In [1]:
#PigLab4NoteBook
import pandas as pd

! pip install pandas
! pip install openpyxl
! pip install rdflib
! pip install pyspark
import rdflib
import hashlib
import numpy as np
import urllib.parse
from rdflib import Literal, Namespace, RDF, URIRef, Graph, RDFS, OWL
from rdflib.namespace import FOAF, XSD
from rdflib.plugins.sparql import prepareQuery
from pyspark.sql.functions import when, col, lit

## First data upload

In [2]:
# Define the file paths
files = ['Lab4/Copia de Dades projecte DECIDE 20230724.csv']

# Load the data into Pandas dataframes
dfs = []
for file in files:
    df = pd.read_csv(file, sep=";")
    dfs.append(df)

df = dfs[0]


In [3]:
df.head(50)

,Identificación,Fecha,Provincia,Patógeno,Resultado,Edad,Tipo Edad,Material,Animal,Categoría
0,1,01/07/2020,NaN,M. hyopneumoniae,NEG,0.0,NaN,T-Pulmón,NaN,NaN
1,1,01/07/2020,NaN,Influenza,NEG,0.0,NaN,T-Pulmón,NaN,NaN
2,2,01/07/2020,NaN,M. hyopneumoniae,POS,0.0,NaN,T-Pulmón,NaN,NaN
3,2,01/07/2020,NaN,Influenza,NEG,0.0,NaN,T-Pulmón,NaN,NaN
4,3,01/07/2020,NaN,M. hyopneumoniae,POS,0.0,NaN,T-Pulmón,NaN,NaN
5,3,01/07/2020,NaN,Influenza,POS,0.0,NaN,T-Pulmón,NaN,NaN
6,4,01/07/2020,NaN,M. hyopneumoniae,POS,0.0,NaN,T-Pulmón,NaN,NaN
7,4,01/07/2020,NaN,Influenza,NEG,0.0,NaN,T-Pulmón,NaN,NaN
8,5,01/07/2020,NaN,Influenza,NEG,0.0,NaN,F-Fluido Oral,NaN,NaN
9,5,01/07/2020,NaN,Influenza,POS,0.0,NaN,F-Fluido Oral,NaN,NaN


In [4]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df['Edad'] = df['Edad'].apply(clean_text)
df['Tipo Edad'] = df['Tipo Edad'].apply(clean_text)
df['Animal'] = df['Animal'].apply(clean_text)
df['Categoría'] = df['Categoría'].apply(clean_text)
df['Provincia'] = df['Provincia'].apply(clean_text)

print(df['Provincia'].unique())

[None 'girona' 'granada' 'huesca' 'sevilla' 'avila' 'lleida' 'zaragoza'
 'huelva' 'barcelona' 'segovia' 'toledo' 'malaga' 'murcia' 'badajoz'
 'alicante' 'pontevedra' 'soria' 'salamanca' 'jaen' 'albacete' 'zamora'
 'almeria' 'tarragona' 'valladolid' 'madrid' 'valencia' 'navarra'
 'castellon' 'orense' 'burgos' 'palencia' 'la coruna']


In [5]:
#Here Age, Ageunit and Farmtype are PigStage


df = df.rename(columns={
    'Identificación': 'SampleNumber',
    'Fecha': 'Date',
    'Provincia': 'Province',
    'Patógeno': 'Pathogen',
    'Resultado': 'Result',
    'Edad': 'Age', #Age of animal
    'Tipo Edad': 'AgeUnit', #represents the unit used to measure the age of animals (e.g, Weeks, Month,year)
    'Material': 'SampleType',
    'Animal': 'BreedType', 
    'Categoría':'FarmType',#refers to the productive stage (or the farming period) of the animals (fattening, nursery, weaned pigs, etc.)
})

df['DiagnosticTest'] = 'PCR'
df['Country'] = 'Spain'
df['Breed'] = 'Meat'
df["Lab_Reference"] = "4"



# Combine 'Age' and 'Age Unit' columns into a new column 'Combined_Age'
df['Age'] = df['Age'].replace(0.0, None)
#df['AgeUnit'] = df['AgeUnit'].fillna('Missing')

df['Age'] = df['Age'].astype(str) + ' ' + df['AgeUnit'].astype(str)

# Replace 'Missing' if both 'Age' and 'AgeUnit' are missing
#df['Age'] = df.apply(lambda row: 'Missing' if row['Age'] == None else row['Age'], axis=1).fillna(None)
# Replace to None if both 'Age' and 'AgeUnit' are missing
df['Age'] = df['Age'].apply(lambda x: None if x == 'Missing' else x)

# Drop the original 'Edad' and 'Tipo Edad' columns if needed
df = df.drop(['AgeUnit'], axis=1)


In [6]:
#Annotation Mapping

#Pathogen and Sample Mapping and Annotations
PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza" : "Swine influenza virus",
}

df["Pathogen"] = df["Pathogen"].map(PathogenMapping).fillna("Missing")

SampleTypeMapping = {
    'T-Pulmón': 'Tissue',
    'F-Fluido Oral': 'Fluid',
    'F-Líquido traqueobronquial': 'BAL',
    'T-Pulmón y bronquio': 'Mixed',
    'T-Pulmón y linfonodo': 'Mixed',
    'T-Pulmón y tonsila': 'Mixed',
    'Escobillones': 'Swab',
    'Tejidos': 'Mixed',
    'T-Pulmón y timo': 'Mixed',
    'Fluídos del procesado': 'Fluid',
    'F-Líquido traqueal': 'BAL',
    'T-Lenguas': 'Tissue',
    'E-Nasales': 'Swab',
    'T-Pulmón, linfonodo y tonsila': 'Mixed',
    'T-Testículos': 'Tissue',
    'T-Pulmón y corazón': 'Mixed',
    'E-Pulmón': 'Swab',
    'T-Pulmón,hígado,riñón,bazo,int': 'Mixed',
    'FTA': 'Other',
    'T-Pulmón e hígado': 'Mixed',
    'E-Vaginales': 'Swab',
    'E-Escobillón de saliva': 'Swab',
    'T-Linfonodo': 'Tissue',
    'Fluídos corp.': 'Fluid',
    'E-Escobillón abdominal': 'Swab',
    'T-Pulmón y bazo': 'Mixed',
    'Sangre': 'Blood',
    'T-Cordón umbilical': 'Tissue',
    'Heces': 'Faeces',
    'F-Líquidos de procesado': 'Fluid',
    'T-Colas': 'Tissue',
    'T-Testículos y colas': 'Mixed',
    'T-Intestino': 'Tissue',
    'T-Pulmón, timo y linfonodo': 'Mixed',
    'F-Exudados': 'Fluid',
    'T-Pulmón, hígado y tonsila': 'Mixed',
    'T-Pulmón y cordón umbilical': 'Mixed',
    'Placa sembrada': 'Other',
    'Cepas': 'Other',
    'F-Semen': 'Fluid',
    'Vacuna': 'Other',
    'Líquido Preservante': 'Other',
    'E-Rectales': 'Swab',
    'F-Fluídos del procesado': 'Fluid',
    'RNA': 'Other',
    'Inóculo': 'Other',
    'F-Líquido pericárdico': 'Fluid',
    'F-Líquido torácico': 'Fluid',
    'T-Vísceras': 'Mixed',
    'FTA ELUTE': 'Other',
    'T-Rabo': 'Tissue',
    'Contenido intestinal': 'Faeces',
    'F-Secreción nasal': 'Fluid',
    'T-Ovario': 'Tissue',
    'T-Placenta': 'Tissue',
    'T-Embrión': 'Tissue',
    'T-Feto': 'Tissue',
    'T-Pulmón,bazo y linfonodo': 'Mixed',
    'Suero': 'Blood',
    'E-Intestino': 'Swab',
    'T-Lenguas y colas': 'Mixed',
    'E-Bronquio': 'Swab',
    'T-Pulmón,hígado,intestin, bazo': 'Mixed',
    'E-Tráquea': 'Swab',
    'Escobillón Digestivo': 'Swab',
    'T-Lenguas y testículos': 'Mixed',
    'T-Lenguas, colas y testículos': 'Mixed',
    'F-Líquidos': 'Fluid',
    'F-Líquido amniótico': 'Fluid',
    'E-Cavidad torácica': 'Swab',
    'T-Feto y placenta': 'Mixed',
    'FTA Digestivo': 'Other',
    ' ': 'Missing',
}

# Ex

# Map the SampleType column using the defined mapping
df["SampleType"] = df["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "NEG": "0",
    "POS": "1",
}
df["Result"] = df["Result"].map(resultMapping).fillna("Missing")


# Province annotation  mapping
ProvinceMapping = {
    'alava': 'Alava',
    'albacete': 'Albacete',
    'alicante': 'Alicante',
    'almeria': 'Almeria',
    'avila': 'Avila',
    'badajoz': 'Badajoz',
    'baleares': 'IslasBaleares',
    'barcelona': 'Barcelona',
    'burgos': 'Burgos',
    'caceres': 'Caceres',
    'cadiz': 'Cadiz',
    'castellon': 'Castellon',
    'cordoba': 'Cordoba',
    'cuenca': 'Cuenca',
    'girona': 'Girona',
    'granada': 'Granada',
    'huelva': 'Huelva',
    'huesca': 'Huesca',
    'jaen': 'Jaen',
    'la coruna': 'ACoruna',
    'la rioja': 'LaRioja',
    'leon': 'Leon',
    'lleida': 'Lleida',
    'lugo': 'Lugo',
    'madrid': 'Madrid',
    'malaga': 'Malaga',
    'murcia': 'Murcia',
    'navarra': 'Navarre',
    'None': 'Missing',
    'orense': 'Ourense',
    'palencia': 'Palencia',
    'pontevedra': 'Pontevedra',
    'salamanca': 'Salamanca',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'soria': 'Soria',
    'tarragona': 'Tarragona',
    'teruel': 'Teruel',
    'toledo': 'Toledo',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',

}


# Translate provinces in the 'referncia' column using the mapping
df['Province'] = df['Province'].map(ProvinceMapping).fillna('Missing')

# Step 1: Convert 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# Step 2: Format to Year, Month, Day
df['Date'] = df['Date'].dt.strftime("%Y-%m-%d")

In [7]:
#df['AnimalType'] = df1[['BreedType', 'FarmType']].agg(' '.join, axis=1)
#df['AnimalType'] = df[['BreedType', 'FarmType']].astype(str).agg(' '.join, axis=1)
df['AnimalType'] = df[['BreedType', 'FarmType']].agg(lambda x: ' '.join(filter(None, map(str, x.dropna()))), axis=1)

#print(df['AnimalType'].unique())
#Join AnimalType & Age
#df['AnimalType&Age'] = df[['AnimalType', 'Age']].astype(str).agg(' '.join, axis=1)
df['AnimalType&Age'] = df[['AnimalType', 'Age']].agg(lambda x: ' '.join(filter(None, map(str, x.dropna()))), axis=1)

df['AnimalType&Age'] = df['AnimalType&Age'].str.replace('None', '', regex=False).str.strip()

print(df['AnimalType&Age'].unique())


['0.0' 'cerdos final transicion 0.0' '6.0 semanas' '7.0 semanas'
 '11.0 semanas' '12.0 semanas' 'cerdos 0.0' 'cerdos 16.0 semanas'
 'cerdos 10.0 semanas' 'cerdos transicion 10.0 semanas'
 'cerdos 8.0 semanas' 'cerdos engorde 0.0' '10.0 semanas'
 'cerdos 6.0 semanas' 'cerdas primiparas 0.0' 'cerdas multiparas 0.0'
 'cerdas 0.0' 'lechones 35.0 dias' 'lechones 56.0 dias'
 'cerdas nuliparas 0.0' 'cerdos 42.0 dias' 'cerdos 60.0 dias'
 'lechones destetados 4.0 semanas' 'lechones destetados 7.0 semanas'
 '17.0 semanas' 'lechones 5.0 semanas' 'lechones 7.0 semanas'
 'cerdos transicion 0.0' 'lechones 0.0' '90.0 kg'
 'cerdos final transicion 10.0 semanas' 'lechones 4.0 semanas'
 'cerdas fut. reproduct. 0.0' 'cerdos engorde 23.0 dias'
 'cerdas reproductoras 0.0' 'cerdos 9.0 semanas' 'cerdos 3.0 meses'
 'cerdos 4.0 meses' 'cerdos 7.0 meses' 'cerdos 30.0 kg' 'cerdos 50.0 kg'
 'cerdos 100.0 kg' 'cerdos transicion 5.0 semanas'
 'cerdos engorde 11.0 semanas' 'cerdos final transicion 8.0 semanas'
 'cer

In [8]:
PigAgeMapping = {
'0': 'Missing',
'cerdos final transicion 0.0': 'Nursery',
'6.0 semanas': 'Nursery',
'7.0 semanas': 'Nursery',
'11.0 semanas': 'Finishing',
'12.0 semanas': 'Finishing',
'cerdos 0.0': 'Finishing',
'cerdos 16.0 semanas': 'Finishing',
'cerdos 10.0 semanas': 'Nursery',
'cerdos transicion 10.0 semanas': 'Nursery',
'cerdos 8.0 semanas': 'Nursery',
'cerdos engorde 0.0': 'Finishing',
'10.0 semanas': 'Nursery',
'cerdos 6.0 semanas': 'Nursery',
'cerdas primiparas 0.0': 'Sow',
'cerdas multiparas 0.0': 'Sow',
'cerdas 0.0': 'Sow',
'lechones 35.0 dias': 'Nursery',
'lechones 56.0 dias': 'Nursery',
'cerdas nuliparas 0.0': 'Gilt',
'cerdos 42.0 dias': 'Nursery',
'cerdos 60.0 dias': 'Nursery',
'lechones destetados 4.0 semanas': 'Weaning',
'lechones destetados 7.0 semanas': 'Nursery',
'17.0 semanas': 'Finishing',
'lechones 5.0 semanas': 'Nursery',
'lechones 7.0 semanas': 'Nursery',
'cerdos transicion 0.0': 'Nursery',
'lechones 0.0': 'Suckling',
'90.0 kg': 'Finishing',
'cerdos final transicion 10.0 semanas': 'Nursery',
'lechones 4.0 semanas': 'Weaning',
'cerdas fut. reproduct. 0.0': 'Gilt',
'cerdos engorde 23.0 dias': 'Finishing',
'cerdas reproductoras 0.0': 'Sow',
'cerdos 9.0 semanas': 'Nursery',
'cerdos 3.0 meses': 'Finishing',
'cerdos 4.0 meses': 'Finishing',
'cerdos 7.0 meses': 'Finishing',
'cerdos 30.0 kg': 'Finishing',
'cerdos 50.0 kg': 'Finishing',
'cerdos 100.0 kg': 'Finishing',
'cerdos transicion 5.0 semanas': 'Nursery',
'cerdos engorde 11.0 semanas': 'Finishing',
'cerdos final transicion 8.0 semanas': 'Nursery',
'cerdos final transicion 60.0 dias': 'Nursery',
'cerdos final transicion 30.0 dias': 'Nursery',
'4.0 semanas': 'Weaning',
'cerdos transicion 8.0 semanas': 'Nursery',
'3.0 semanas': 'Weaning',
'cerdos 40.0 kg': 'Finishing',
'cerdos engorde 17.0 semanas': 'Finishing',
'lechones 25.0 dias': 'Weaning',
'cerdos engorde 90.0 kg': 'Finishing',
'cerdos engorde 80.0 kg': 'Finishing',
'lechones 21.0 dias': 'Weaning',
'cerdos destetados 49.0 dias': 'Nursery',
'cerdos engorde 4.0 meses': 'Finishing',
'cerdos transicion 6.0 semanas': 'Nursery',
'4.0 meses': 'Finishing',
'5.0 meses': 'Finishing',
'9.0 semanas': 'Nursery',
'cerdos 5.0 semanas': 'Nursery',
'21.0 dias': 'Weaning',
'45.0 dias': 'Nursery',
'cerdos final transicion': 'Nursery',
'lechones 11.0 semanas': 'Nursery',
'lechones destetados 0.0': 'Weaning',
'25.0 dias': 'Weaning',
'cerdos 31.0 dias': 'Nursery',
'cerdas gestacion 0.0': 'Sow',
'16.0 semanas': 'Finishing',
'lechones 9.0 semanas': 'Nursery',
'lechones lactacion 3.0 semanas': 'Suckling',
'cerdos destetados 9.0 semanas': 'Nursery',
'20.0 kg': 'Finishing',
'40.0 kg': 'Finishing',
'100.0 kg': 'Finishing',
'5.0 semanas': 'Nursery',
'cerdos 13.0 semanas': 'Finishing',
'50.0 dias': 'Nursery',
'70.0 dias': 'Nursery',
'90.0 dias': 'Finishing',
'110.0 dias': 'Finishing',
'cerdas reposicion 0.0': 'Gilt',
'verracos 0.0': 'Boar',
'8.0 semanas': 'Nursery',
'17.0 kg': 'Nursery',
'21.0 kg': 'Finishing',
'26.0 kg': 'Finishing',
'35.0 kg': 'Finishing',
'lechones lactacion 6.0 semanas': 'Nursery',
'lechones lactacion 9.0 semanas': 'Nursery',
'cerdos 7.0 semanas': 'Nursery',
'cerdos destetados 0.0': 'Weaning',
'cerdos 4.0 semanas': 'Weaning',
'cerdos 11.0 semanas': 'Finishing',
'cerdos engorde 12.0 semanas': 'Finishing',
'cerdos engorde 6.0 semanas': 'Finishing',
'cerdos engorde 9.0 semanas': 'Finishing',
'lechones 6.0 semanas': 'Nursery',
'cerdos 15.0 semanas': 'Finishing',
'cerdos 24.0 semanas': 'Finishing',
'3.0 meses': 'Finishing',
'cerdos 40.0 dias': 'Sow',
'cerdos 67.0 dias': 'Nursery',
'cerdos 28.0 dias': 'Nursery',
'lechones destetados 9.0 semanas': 'Weaning',
'cerdos engorde 3.0 meses': 'Finishing',
'cerdos 33.0 dias': 'Nursery',
'cerdos 47.0 dias': 'Nursery',
'cerdos 18.0 kg': 'Nursery',
'cerdos engorde 16.0 semanas': 'Finishing',
'cerdos 41.0 dias': 'Nursery',
'cerdos 62.0 dias': 'Nursery',
'cerdas nuliparas 4.0 meses': 'Gilt',
'cerdas nuliparas 7.0 meses': 'Gilt',
'lechones 10.0 semanas': 'Nursery',
'175.0 dias': 'Finishing',
'135.0 dias': 'Finishing',
'cerdos 2.0 meses': 'Nursery',
'lechones 23.0 dias': 'Weaning',
'lechones 58.0 dias': 'Nursery',
'6.0 meses': 'Finishing',
'18.0 semanas': 'Finishing',
'cerdas 30.0 semanas': 'Gilt',
'cerdas jovenes 0.0': 'Gilt',
'10.0 meses': 'Finishing',
'cerdas 5.0 meses': 'Gilt',
'cerdos transicion 9.0 semanas': 'Nursery',
'cerdos engorde 14.0 semanas': 'Finishing',
'cerdos engorde 10.0 semanas': 'Finishing',
'cerdos 18.0 semanas': 'Finishing',
'8.0 kg': 'Nursery',
'12.0 kg': 'Nursery',
'18.0 kg': 'Nursery',
'110.0 kg': 'Finishing',
'cerdos engorde 50.0 kg': 'Finishing',
'cerdos engorde 70.0 kg': 'Finishing',
'30.0 dias': 'Nursery',
'40.0 dias': 'Nursery',
'lechones 8.0 semanas': 'Nursery',
'cerdas reposicion 5.0 meses': 'Gilt',
'60.0 dias': 'Nursery',
'lechones 3.0 semanas': 'Weaning',
'cerdos destetados 7.0 semanas': 'Nursery',
'lechones 12.0 semanas': 'Finishing',
'cerdos 12.0 semanas': 'Finishing',
'cerdos 14.0 semanas': 'Finishing',
'63.0 dias': 'Nursery',
'49.0 dias': 'Nursery',
'15.0 semanas': 'Finishing',
'13.0 semanas': 'Finishing',
'cerdas fut. reproduct. 4.0 meses': 'Gilt',
'cerdas 4.0 semanas': 'Gilt',
'cerdas 6.0 semanas': 'Gilt',
'cerdas 9.0 semanas': 'Gilt',
'cerdos transicion 2.0 semanas': 'Nursery',
'cerdos transicion 1.0 semanas': 'Nursery',
'cerdos transicion 4.0 semanas': 'Nursery',
'cerdos transicion 7.0 semanas': 'Nursery',
'cerdas reposicion 6.0 meses': 'Gilt',
'cerdos engorde 15.0 semanas': 'Finishing',
'42.0 dias': 'Nursery',
'cerdos engorde 25.0 semanas': 'Finishing',
'cerdos 20.0 semanas': 'Finishing',
'cerdos 3.0 semanas': 'Weaning',
'cerdos engorde 18.0 semanas': 'Finishing',
'cerdos 105.0 dias': 'Finishing',
'cerdos 90.0 dias': 'Finishing',
'cerdos 75.0 dias': 'Finishing',
'cerdos 50.0 dias': 'Nursery',
'lechones 45.0 dias': 'Nursery',
'cerdos transicion 3.0 semanas': 'Nursery',
'lechones destetados 32.0 dias': 'Nursery',
'cerdos engorde 90.0 dias': 'Finishing',
'cerdas reposicion 8.0 meses': 'Gilt',
'cerdos engorde 40.0 kg': 'Finishing',
'cerdos 80.0 kg': 'Finishing',
'cerdos transicion 12.0 semanas': 'Nursery',
'cerdos transicion 30.0 dias': 'Nursery',
'cerdos transicion 60.0 dias': 'Nursery',
'cerdos transicion 45.0 dias': 'Nursery',
'lechones destetados 10.0 semanas': 'Nursery',
'cerdos engorde 13.0 semanas': 'Finishing',
'lechones 37.0 dias': 'Nursery',
'lechones 31.0 dias': 'Nursery',
'lechones 52.0 dias': 'Nursery',
'cerdos 35.0 dias': 'Nursery',
'cerdos engorde 27.0 semanas': 'Finishing',
'cerdos engorde 26.0 semanas': 'Finishing',
'cerdos 25.0 semanas': 'Finishing',
'cerdos 27.0 semanas': 'Finishing',
'cerdos engorde 28.0 semanas': 'Finishing',
'cerdas jovenes 5.0 meses': 'Gilt',
'cerdos engorde 100.0 kg': 'Finishing',
'26.0 semanas': 'Finishing',
'27.0 semanas': 'Finishing',
'25.0 semanas': 'Finishing',
'final transicion 0.0': 'Nursery',
'50.0 kg': 'Finishing',
'cerdos 5.0 meses': 'Finishing',
'cerdos transicion 42.0 dias': 'Nursery',
'cerdos engorde 23.0 semanas': 'Finishing',
'cerdos engorde 30.0 kg': 'Finishing',
'cerdos engorde 60.0 kg': 'Finishing',
'cerdos engorde 7.0 meses': 'Finishing',
'lechones destetados 5.0 semanas': 'Nursery',
'32.0 dias': 'Nursery',
'54.0 dias': 'Nursery',
'lechones 32.0 dias': 'Nursery',
'lechones 63.0 dias': 'Nursery',
'lechones 41.0 dias': 'Nursery',
'cerdos destetados 60.0 kg': 'Finishing',
'cerdos 140.0 dias': 'Finishing',
'cerdos 130.0 dias': 'Finishing',
'cerdas 10.0 meses': 'Gilt',
'cerdas gestacion 12.0 meses': 'Sow',
'cerdas gestacion 14.0 meses': 'Sow',
'cerdos engorde 6.0 meses': 'Finishing',
'lechones 1.0 semanas': 'Suckling',
'lechones 2.0 semanas': 'Suckling',
'lechones engorde 0.0': 'Finishing',
'2.0 meses': 'Nursery',
'8.0 meses': 'Finishing',
'lechones 40.0 dias': 'Nursery',
'cerdos destetados 8.0 semanas': 'Nursery',
'lechones 20.0 kg': 'Finishing',
'lechones lactacion 1.0 semanas': 'Suckling',
'cerdos final transicion 9.0 semanas': 'Nursery',
'cerdos engorde 2.0 meses': 'Finishing',
'cerdos engorde': 'Finishing',
'lechones destetados': 'Weaning',
'cerdos 16.0 kg': 'Nursery',
'cerdos 10.0 meses': 'Finishing',
'cerdos 49.0 dias': 'Nursery',
'lechones lactacion 0.0': 'Suckling',
'56.0 dias': 'Nursery',
'cerdos engorde 25.0 kg': 'Finishing',
'cerdos 10.0 kg': 'Nursery',
'cerdas reposicion 7.0 meses': 'Gilt',
'15.0 dias': 'Suckling',
'35.0 dias': 'Nursery',
'lechones 28.0 dias': 'Weaning',
'lechones destetados 6.0 semanas': 'Nursery',
'cerdos transicion 59.0 dias': 'Nursery',
'cerdos transicion 38.0 dias': 'Nursery',
'cerdos': 'Finishing',
'lechones 70.0 dias': 'Nursery',
'cerdos final transicion 70.0 dias': 'Nursery',
'lechones destetados 8.0 semanas': 'Nursery',
'14.0 semanas': 'Finishing',
'lechones destetados 42.0 dias': 'Nursery',
'lechones destetados 60.0 dias': 'Nursery',
'cerdos 54.0 dias': 'Nursery',
'lechones 48.0 dias': 'Nursery',
'lechones 74.0 dias': 'Nursery',
'cerdos transicion 50.0 dias': 'Nursery',
'lechones destetados 45.0 dias': 'Nursery',
'1.0 semanas': 'Suckling',
'lechones 1.0 dias': 'Suckling',
'lechones 15.0 dias': 'Suckling',
'cerdos transicion 21.0 dias': 'Weaning',
'cerdos transicion 35.0 dias': 'Nursery',
'cerdos transicion 53.0 dias': 'Nursery',
'cerdos transicion 25.0 dias': 'Weaning',
'cerdos 45.0 dias': 'Nursery',
'cerdos 70.0 dias': 'Nursery',
'75.0 dias': 'Finishing',
'65.0 dias': 'Nursery',
'55.0 dias': 'Nursery',
'cerdas 24.0 semanas': 'Finishing',
'cerdas 18.0 semanas': 'Finishing',
'cerdas 15.0 semanas': 'Finishing',
'lechones 10.0 dias': 'Suckling',
'cerdos transicion 11.0 semanas': 'Finishing',
'cerdos engorde 5.0 semanas': 'Nursery',
'cerdos engorde 7.0 semanas': 'Nursery',
'lechones 54.0 dias': 'Nursery',
'cerdos 30.0 semanas': 'Finishing',
'cerdos 30.0 dias': 'Nursery',
'cerdos 55.0 dias': 'Nursery',
'lechones 17.0 kg': 'Nursery',
'cerdos 25.0 dias': 'Weaning',
'cerdos engorde 4.0 semanas': 'Weaning',
'28.0 dias': 'Weaning',
'cerdos 29.0 semanas': 'Finishing',
'cerdos 85.0 dias': 'Nursery',
'cerdos 65.0 dias': 'Nursery',
'cerdos engorde 5.0 meses': 'Finishing',
'lechones 20.0 dias': 'Suckling',
'cerdos final transicion 12.0 semanas': 'Finishing',
'28.0 semanas': 'Finishing',
'lechones destetados 3.0 semanas': 'Weaning',
'cerdas primiparas 6.0 meses': 'Gilt',
'cerdos 100.0 dias': 'Finishing',
'cerdos destetados 40.0 dias': 'Nursery',
'cerdos destetados 60.0 dias': 'Nursery',
'52.0 dias': 'Nursery',
'cerdos engorde 20.0 semanas': 'Finishing',
'21.0 semanas': 'Finishing',
'19.0 semanas': 'Finishing',
'lechones destetados 32.0 semanas': 'Nursery',
'cerdos 6.0 meses': 'Finishing',
'lechones destetados 38.0 dias': 'Nursery',
'lechones destetados 67.0 dias': 'Nursery',
'34.0 semanas': 'Finishing',
'cerdos engorde 22.0 semanas': 'Finishing',
'lechones 17.0 dias': 'Suckling',
'lechones 14.0 dias': 'Suckling',
'lechones 11.0 dias': 'Suckling',
'cerdos engorde 21.0 semanas': 'Finishing',
'7.0 meses': 'Finishing',
'lechones 60.0 dias': 'Nursery',
'cerdos 1.0 semanas': 'Suckling',
'cerdos transicion 70.0 dias': 'Nursery',
'cerdos 1.0 meses': 'Weaning',
'lechones nuliparas 60.0 dias': 'Other',
'lechones nacido debil 0.0': 'Suckling',
'lechones 18.0 dias': 'Suckling',
'cerdas nuliparas 1.0 semanas': 'Gilt',
'cerdas nuliparas 40.0 kg': 'Gilt',
'cerdas nuliparas 6.0 meses': 'Gilt',
'cerdas final transicion 0.0': 'Nursery',
'cerdos final transicion 5.0 semanas': 'Nursery',
'cerdos final transicion 7.0 semanas': 'Nursery',
'cerdos final transicion 6.0 semanas': 'Nursery',
'cerdas reproductoras 9.0 semanas': 'Gilt',
'lechones 2.0': 'Suckling',
'cerdos engorde 8.0 semanas': 'Nursery',
'lechones 4.0 dias': 'Suckling',
'verracos 5.0 semanas': 'Boar',
'lechones 3.0 dias': 'Suckling',
'cerdas nuliparas 2.0 meses': 'Gilt',
'cerdas nuliparas 3.0 meses': 'Gilt',
'cerdas nuliparas 90.0 dias': 'Gilt',
'cerdas nuliparas 120.0 dias': 'Gilt',
'cerdas nuliparas 150.0 dias': 'Gilt',
'cerdos final transicion 4.0 semanas': 'Nursery',
'cerdas 7.0 meses': 'Gilt',
'lechones': 'Suckling',
'cerdos transicion 49.0 dias': 'Nursery',
'cerdas 8.0 meses': 'Gilt',
'cerdos 60.0 kg': 'Finishing',
'cerdas nuliparas 8.0 meses': 'Gilt',
'cerdas 3.0 semanas': 'Gilt',
'cerdas 5.0 semanas': 'Gilt',
'cerdos engorde 32.0 semanas': 'Finishing',
'lechones lactantes 0.0': 'Suckling',
'cerdas reposicion 60.0 kg': 'Gilt',
'cerdas reposicion 40.0 kg': 'Gilt',
'cerdas reposicion 180.0 kg': 'Gilt',
'cerdos transicion': 'Nursery',
'cerdas 1.0 gr. partos': 'Sow',
'cerdas nuliparas 13.0 semanas': 'Gilt',
'cerdas nuliparas 22.0 semanas': 'Gilt',
'cerdas nuliparas 27.0 semanas': 'Gilt',
'lechones lactantes 21.0 dias': 'Suckling',
'lechones lactantes 3.0 semanas': 'Weaning',
'lechones 27.0 dias': 'Weaning',
'lechones lactantes 24.0 dias': 'Suckling',
'lechones 24.0 dias': 'Weaning',
'cerdas nuliparas 180.0 dias': 'Gilt',
'cerdas nuliparas 220.0 dias': 'Gilt',
'cerdas nuliparas 200.0 dias': 'Gilt',
'lechones lactantes 18.0 dias': 'Suckling',
'cerdas nuliparas 160.0 dias': 'Gilt',
'cerdos final transicion 11.0 semanas': 'Finishing',
'cerdas transicion 6.0 semanas': 'Nursery',
'cerdas 24.0 meses': 'Sow',
'lechones transicion 0.0': 'Nursery',
'cerdos final transicion 75.0 dias': 'Finishing',
'lechones 5.0 dias': 'Suckling',
'lechones 2.0 dias': 'Suckling',
'lechones destetados 21.0 dias': 'Weaning',
'lechones 34.0 dias': 'Nursery',
'lechones 50.0 dias': 'Nursery',
'cerdos transicion 2.0 meses': 'Nursery',
'lechones destetados 18.0 dias': 'Suckling',
'cerdas reposicion 9.0 meses': 'Gilt',
'lechones 30.0 dias': 'Nursery',
'lechones lactacion 21.0 dias': 'Weaning',
'lechones lactacion 10.0 dias': 'Suckling',
'cerdos transicion 65.0 dias': 'Nursery',
'cerdas nuliparas 4.0 semanas': 'Gilt',
'cerdas nuliparas 6.0 semanas': 'Gilt',
'lechones 7.0 dias': 'Suckling',
'cerdos 17.0 semanas': 'Finishing',
'lechones destetados 2.0 semanas': 'Weaning',
'cerdas 8.0 semanas': 'Gilt',
'cerdos engorde 30.0 semanas': 'Finishing',
'cerdas lactantes 0.0': 'Sow',
'lechones 26.0 dias': 'Weaning',
'cerdas reproductoras 24.0 meses': 'Sow',
'cerdos 14.0 dias': 'Suckling',
'lechones nacido debil 2.0 semanas': 'Suckling',
'lechones nacido debil 3.0 semanas': 'Weaning',
'cerdos engorde 24.0 semanas': 'Finishing',
'cerdas nuliparas 8.0 semanas': 'Gilt',
'cerdos 21.0 dias': 'Weaning',
'cerdas 7.0 semanas': 'Gilt',
'lechones destetados 2.0 dias': 'Suckling',
'cerdos 15.0 kg': 'Nursery',
'verracos nuliparas 0.0': 'Other',
'lechones 2.0 meses': 'Nursery',
'cerdas nuliparas': 'Gilt',
'cerdos destetados 10.0 semanas': 'Nursery',
'lechones 49.0 dias': 'Nursery',
'cerdos 10.0 dias': 'Suckling',
'cerdas reproductoras 40.0 kg': 'Gilt',
'cerdas nuliparas 7.0 semanas': 'Gilt',
'cerdas nuliparas 5.0 semanas': 'Gilt',
'cerdos 19.0 semanas': 'Finishing',
'cerdos transicion 16.0 semanas': 'Finishing',
'lechones final transicion 12.0 semanas': 'Finishing',
'cerdos final transicion 90.0 kg': 'Finishing',
'cerdos 20.0 kg': 'Finishing',
'cerdos 80.0 dias': 'Finishing',
'cerdas  semanas': 'Sow',
'cerdas nuliparas 28.0 dias': 'Gilt',
'lechones 0.0 semanas': 'Suckling',
'lechones 3.0 meses': 'Weaning',
'lechones 0.0 dias': 'Suckling',
'cerdas reposicion 100.0 kg': 'Gilt',
'cerdas reposicion 80.0 kg': 'Gilt',
'cerdos destetados 5.0 semanas': 'Nursery',
'cerdos 38.0 dias': 'Nursery',
'cerdas nuliparas 100.0 kg': 'Gilt',
'lechones destetados 28.0 dias': 'Weaning',
'lechones destetados 24.0 dias': 'Weaning',
'cerdos transicion 10.0 dias': 'Suckling',
'cerdos transicion 17.0 dias': 'Suckling',
'cerdos transicion 24.0 dias': 'Weaning',
'cerdas fut. reproduct. 8.0 meses': 'Gilt',
'cerdos engorde 7.0 dias': 'Suckling',
'cerdos final transicion 3.0 semanas': 'Nursery',
'cerdas primiparas 11.0 meses': 'Sow',
'lechones 21.0 kg': 'Finishing',
'lechones lactacion 4.0 semanas': 'Weaning',
'cerdas 12.0 meses': 'Sow',
'cerdos 22.0 semanas': 'Finishing',
'cerdos engorde 150.0 dias': 'Finishing',
'cerdos transicion 24.0 semanas': 'Finishing',
'lechones 19.0 semanas': 'Finishing',
'lechones 22.0 semanas': 'Finishing',
'cerdos 2.0 semanas': 'Suckling',
'cerdos 21.0 semanas': 'Finishing',
'cerdos destetados 28.0 dias': 'Weaning',
'cerdos 43.0 dias': 'Nursery',
'cerdos 46.0 dias': 'Nursery',
'cerdos 2.0 dias': 'Suckling',
'cerdos 6.0': 'Other',
'lechones 90.0 dias': 'Finishing',
'lechones 110.0 dias': 'Finishing',
'lechones lactacion 15.0 dias': 'Suckling',
'cerdas reposicion 28.0 dias': 'Gilt',
'cerdas': 'Sow',
'cerdas 2.0 partos': 'Sow',
'cerdos 0.0 dias': 'Other',
'cerdos transicion 28.0 dias': 'Weaning',
'cerdos transicion 41.0 dias': 'Nursery',
'cerdos transicion 63.0 dias': 'Nursery',
'cerdos 200.0 dias': 'Finishing',
'lechones lactacion 7.0 dias': 'Suckling',
'lechones lactacion 22.0 dias': 'Weaning',
'lechones 22.0 dias': 'Weaning',
'cerdos transicion 7.0 dias': 'Suckling',
'cerdas 2.0 gr. partos': 'Sow',
'cerdas 4.0 gr. partos': 'Sow',
'cerdas 6.0 gr. partos': 'Sow',
'lechones 15.0 kg': 'Nursery',
'lechones 25.0 kg': 'Finishing',
'cerdas nuliparas 0.0 partos': 'Gilt',
'cerdas 5.0 gr. partos': 'Sow',
'lechones 14.0 semanas': 'Finishing',
'cerdas 18.0 meses': 'Sow',
'cerdas 11.0 semanas': 'Gilt',
'verracos 180.0 dias': 'Boar',
'cerdas 180.0 dias': 'Gilt',
'cerdos engorde 15.0': 'Finishing',
'cerdos transicion 54.0 dias': 'Nursery',
'cerdas reposicion 170.0 dias': 'Gilt',
'lechones 1.0 partos': 'Other',
'lechones 2.0 partos': 'Other',
'lechones 3.0 partos': 'Other',
'lechones 4.0 partos': 'Other',
'lechones 5.0 partos': 'Other',
'cerdas 4.0 meses': 'Gilt',
'lechones nuliparas 0.0': 'Other',
'cerdas 1.0 semanas': 'Gilt',
'cerdas 48.0 meses': 'Sow',
'lechones destetados 15.0 dias': 'Suckling',
'lechones destetados 30.0 dias': 'Nursery',
'cerdos 3.0 dias': 'Suckling',
'lechones 9.0 dias': 'Suckling',
'cerdas multiparas 4.0 semanas': 'Sow',
'cerdas multiparas 5.0 semanas': 'Sow',
'lechones 6.0 dias': 'Suckling',
'cerdos transicion 9.0 meses': 'Finishing',
'lechones 36.0 dias': 'Nursery',
'cerdas nuliparas 3.0 semanas': 'Gilt',
'cerdas multiparas 3.0 semanas': 'Sow',
'lechones 16.0 semanas': 'Finishing',
'lechones lactacion 5.0 dias': 'Suckling',
'lechones 44.0 dias': 'Nursery',
'cerdas 28.0 semanas': 'Gilt',
'cerdos engorde 26.0 dias': 'Weaning',
'lechones 42.0 dias': 'Nursery',
'lechones 64.0 dias': 'Nursery',
'cerdos 20.0 dias': 'Nursery',
'lechones lactacion 1.0 dias': 'Suckling',
'verracos engorde 0.0': 'Boar',
'lechones 12.0 dias': 'Suckling',
'cerdos transicion 56.0 dias': 'Nursery',
'cerdos transicion 66.0 dias': 'Nursery',
'cerdas 2.0 meses': 'Gilt',
'lechones reproductoras 24.0 dias': 'Weaning',
'cerdas reproductoras 20.0 dias': 'Gilt',
'cerdas reproductoras 40.0 dias': 'Gilt',
'cerdas reproductoras 60.0 dias': 'Gilt',
'cerdos transicion 14.0 semanas': 'Finishing',
'lechones 13.0 semanas': 'Finishing',
'cerdas nuliparas 5.0 meses': 'Gilt',
'cerdos 57.0 dias': 'Nursery',
'cerdos 23.0 dias': 'Weaning',
'lechones 29.0 dias': 'Nursery',
'cerdos destetados 6.0 semanas': 'Nursery',
'verracos destetados 0.0': 'Weaning',
'lechones 8.0 dias': 'Suckling',
'cerdos 56.0 dias': 'Nursery',
'lechones 77.0 dias': 'Finishing',
'lechones 84.0 dias': 'Finishing',
'lechones  dias': 'Suckling',
'cerdos  semanas': 'Other',
'lechones destetados 11.0 semanas': 'Finishing',
'lechones 17.0': 'Suckling',
'lechones 18.0': 'Suckling',
'lechones 20.0': 'Suckling',
'lechones 1.0': 'Suckling',
'lechones lactacion 25.0 dias': 'Weaning',
'cerdos destetados 55.0 dias': 'Nursery',
'cerdos 26.0 semanas': 'Finishing',
'lechones 19.0 dias': 'Suckling',
'cerdos 120.0 dias': 'Finishing',
'cerdos 160.0 dias': 'Finishing',
'cerdos 180.0 dias': 'Finishing',
'cerdos transicion 40.0 dias': 'Nursery',
'cerdas fut. reproduct. 10.0 meses': 'Gilt',
'cerdos transicion 150.0 dias': 'Finishing',
'cerdos transicion 130.0 dias': 'Finishing',
'cerdos transicion 115.0 dias': 'Finishing',
'cerdos transicion 94.0 dias': 'Finishing',
'lechones 70.0 kg': 'Finishing',
'lechones final transicion 3.0 semanas': 'Suckling',
'lechones 8.0 meses': 'Finishing',
'cerdas 6.0 meses': 'Gilt',
'lechones lactacion 2.0 semanas': 'Suckling',
'cerdos 10.0': 'Other',
'lechones destetados 26.0 dias': 'Weaning',
'cerdos transicion 58.0 dias': 'Nursery',
'cerdos 24.0 dias': 'Weaning',
'cerdos engorde 22.0 kg': 'Finishing',
'cerdos engorde 110.0 kg': 'Finishing',
'cerdas nuliparas 28.0 semanas': 'Gilt',
'lechones nacido debil 0.0 dias': 'Suckling',
'cerdas nacido debil 0.0': 'Sow',
'lechones 75.0 dias': 'Finishing',
'lechones lactacion 2.0 dias': 'Suckling',
'cerdas primiparas 2.0 dias': 'Sow',
'cerdas multiparas 2.0 dias': 'Sow',
'lechones multiparas 0.0': 'Other',
'cerdos engorde 22.0 dias': 'Weaning',
'cerdas fut. reproduct. 50.0 kg': 'Gilt',
'verracos 4.0 meses': 'Boar',
'lechones 72.0 dias': 'Finishing',
'cerdos final transicion 4.0': 'Nursery',
'cerdos 1.0 dias': 'Suckling',
'cerdos 7.0 dias': 'Suckling',
'cerdos 23.0 semanas': 'Finishing',
'cerdas primiparas 8.0 meses': 'Sow',
'cerdos destetados 25.0 dias': 'Weaning',
'lechones destetados 8.0 dias': 'Weaning',
'cerdos multiparas 0.0': 'Sow',
'cerdos nuliparas 0.0': 'Gilt',
'lechones nacido debil 4.0 semanas': 'Weaning',
'cerdos transicion 15.0 semanas': 'Finishing',
'cerdos 98.0 dias': 'Finishing',
'cerdas reposicion 180.0 dias': 'Gilt',
'lechones 6.0 kg': 'Weaning',
'cerdas reposicion 100.0 dias': 'Gilt',
'cerdas reposicion 120.0 dias': 'Gilt',
'cerdos final transicion 50.0 dias': 'Nursery',
'cerdos 150.0 dias': 'Finishing',
'cerdos 5.0 dias': 'Suckling',
'lechones destetados 10.0 dias': 'Suckling',
'lechones nacido debil 2.0 dias': 'Suckling',
'lechones destetados 43.0 dias': 'Nursery',
'cerdas fut. reproduct. 80.0 kg': 'Gilt',
'cerdas reposicion 15.0 semanas': 'Gilt',
'cerdas reposicion 18.0 semanas': 'Gilt',
'cerdos 29.0 dias': 'Nursery',
'lechones 66.0 dias': 'Nursery',
'cerdos transicion 80.0 dias': 'Finishing',
'lechones 62.0 dias': 'Nursery',
'cerdos 61.0 dias': 'Nursery',
'lechones final transicion 0.0': 'Nursery',
'cerdos final transicion 90.0 dias': 'Finishing',
'cerdos nacido debil 0.0': 'Suckling',
'lechones 40.0 kg': 'Finishing',
'lechones primiparas 4.0 semanas': 'Other',
'cerdos transicion 71.0 dias': 'Finishing',
'lechones 16.0 dias': 'Suckling',
'cerdas 3.0 gr. partos': 'Sow',
'cerdas 7.0 gr. partos': 'Sow',
'cerdos transicion 5.0 dias': 'Suckling',
'lechones 73.0 dias': 'Finishing',
'cerdas fut. reproduct. 5.0 semanas': 'Gilt',
'cerdos engorde 19.0 semanas': 'Finishing',
'lechones destetados 1.0 semanas': 'Suckling',
'verracos': 'Boar',
'cerdos engorde 17.0 kg': 'Finishing',
'cerdos transicion 55.0 dias': 'Nursery',
'lechones 30.0 kg': 'Finishing',
'lechones lactacion 23.0 dias': 'Weaning',
'lechones 25.0 semanas': 'Finishing',
'lechones 7.0 kg': 'Weaning',
'cerdos 15.0 dias': 'Suckling',
'reproductoras 0.0': 'Sow',
'lechones 26.0 semanas': 'Finishing',
'lechones 27.0 semanas': 'Finishing',
'lechones 23.0 semanas': 'Finishing',
'lechones 24.0 semanas': 'Finishing',
'lechones 53.0 dias': 'Nursery',
'lechones nacido debil 15.0 dias': 'Suckling',
'lechones 1.0 meses': 'Weaning',
'cerdas reposicion 200.0 dias': 'Gilt',
'cerdas reposicion 187.0 dias': 'Gilt',
'cerdas reposicion 173.0 dias': 'Gilt',
'cerdas reposicion 155.0 dias': 'Gilt',
'cerdos engorde 20.0 kg': 'Finishing',
'cerdos 90.0 kg': 'Finishing',
'lechones 41.0 semanas': 'Nursery',
'lechones 42.0 semanas': 'Nursery',
'cerdas reposicion 120.0 kg': 'Gilt',
'cerdas reposicion 70.0 kg': 'Gilt',
'cerdas reproductoras': 'Sow',
'cerdos 8.0 meses': 'Finishing',
'cerdos 140.0 kg': 'Finishing',
'cerdos 135.0 kg': 'Finishing',
'cerdos 125.0 kg': 'Finishing',
'cerdos 110.0 kg': 'Finishing',
'cerdos 105.0 kg': 'Finishing',
'cerdos 93.0 kg': 'Finishing',
'cerdos 70.0 kg': 'Finishing',
'cerdos 65.0 kg': 'Finishing',
'cerdos 55.0 kg': 'Finishing',
'cerdas 25.0 dias': 'Gilt',
'cerdas reposicion 4.0 meses': 'Gilt',
'lechones  semanas': 'Suckling',
'lechones 46.0 dias': 'Nursery',
'cerdos engorde 1.0 semanas': 'Suckling',
'cerdos engorde 35.0 kg': 'Finishing',
'cerdas 21.0 dias': 'Gilt',
'lechones 8.0 kg': 'Nursery',
'lechones destetados 50.0 dias': 'Nursery',
'lechones destetados 70.0 dias': 'Nursery',
'cerdas reposicion 22.0 semanas': 'Gilt',
'cerdas 60.0 kg': 'Gilt',
'lechones 13.0 dias': 'Suckling',
'lechones 20.0 semanas': 'Finishing',
'cerdos 250.0 dias': 'Finishing',
'cerdas nuliparas 33.0 semanas': 'Gilt',
'cerdas nuliparas 20.0 semanas': 'Gilt',
'lechones 19.0 kg': 'Nursery',
'cerdas reposicion 110.0 kg': 'Gilt',
'cerdos 63.0 dias': 'Nursery',
'cerdos transicion 8.0 dias': 'Suckling',
'lechones lactacion': 'Suckling',
'lechones destetados 25.0 dias': 'Weaning',
'lechones lactacion 20.0 dias': 'Suckling',
'lechones nacido debil 1.0 dias': 'Suckling',
'cerdas 1.0 partos': 'Sow',
'cerdas 3.0 partos': 'Sow',
'cerdas 4.0 partos': 'Sow',
'cerdas reproductoras 18.0 meses': 'Sow',
'cerdos transicion 6.0': 'Nursery',
'lechones 4.0': 'Suckling',
'cerdos destetados 21.0 dias': 'Weaning',
'cerdas fut. reproduct. 12.0 semanas': 'Gilt',
'cerdas multiparas 1.0 semanas': 'Sow',
'lechones 55.0 dias': 'Nursery',
'lechones lactacion 18.0 dias': 'Suckling',
'lechones destetados 3.0 dias': 'Suckling',
'lechones destetados 52.0 dias': 'Nursery',
'lechones lactacion 3.0 dias': 'Suckling',
'lechones 33.0 semanas': 'Finishing',
'cerdas 35.0 dias': 'Gilt',
'cerdos destetados 4.0 semanas': 'Weaning',
'cerdos 27.0 dias': 'Weaning',
'cerdos destetados 80.0 dias': 'Finishing',
'cerdos engorde 31.0 semanas': 'Finishing',
'lechones 80.0 dias': 'Finishing',
'cerdos destetados 70.0 dias': 'Nursery',
'cerdas 20.0 meses': 'Gilt',
'cerdos destetados': 'Weaning',
'lechones 65.0 dias': 'Nursery',
'lechones lactacion 30.0 dias': 'Weaning',
'lechones destetados 22.0 dias': 'Weaning',
'cerdos engorde 180.0 dias': 'Finishing',
'cerdos engorde 160.0 dias': 'Finishing',
'cerdos engorde 140.0 dias': 'Finishing',
'cerdos engorde 120.0 dias': 'Finishing',
'lechones destetados 34.0 dias': 'Nursery',
'cerdas 32.0 semanas': 'Gilt',
'lechones 33.0 dias': 'Nursery',
'lechones lactacion 28.0 dias': 'Weaning',
'cerdas reposicion 160.0 dias': 'Gilt',
'cerdas reposicion 130.0 dias': 'Gilt',
'cerdas reposicion 150.0 dias': 'Gilt',
'cerdas jovenes 22.0 dias': 'Gilt',
'cerdas 22.0 dias': 'Gilt',
'cerdas primiparas 23.0 dias': 'Gilt',
'cerdas multiparas 23.0 dias': 'Gilt',
'cerdas 20.0 dias': 'Gilt',
'cerdas 42.0 dias': 'Gilt',
'cerdas 56.0 dias': 'Gilt',
'lechones 47.0 dias': 'Nursery',
    '3.0 dias': 'Suckling',
'2.0 dias': 'Suckling',
'80.0 kg': 'Finishing',
'30.0 semanas': 'Finishing',
'2.0 semanas': 'Suckling',
'20.0 semanas': 'Finishing',
'103.0 dias': 'Finishing',
'3': 'Missing',
'36.0 dias': 'Nursery',
'0.0 semanas': 'Missing',
'1.0 dias': 'Suckling',
'51.0 dias': 'Nursery',
'5.0 dias': 'Suckling',
'180.0 dias': 'Finishing',
'150.0 dias': 'Finishing',
'0.0 dias': 'Missing',
'25.0 kg': 'Finishing',
'70.0 kg': 'Finishing',
'120.0 kg': 'Finishing',
'160.0 dias': 'Finishing',
'39.0 dias': 'Nursery',
'1.0 partos': 'Sow',
'77.0 dias': 'Finishing',
'7.0 dias': 'Suckling',
'20.0 dias': 'Suckling',
'170.0 dias': 'Finishing',
'200.0 dias': 'Finishing',
'140.0 dias': 'Finishing',
'120.0 dias': 'Finishing',
'6.0 dias': 'Suckling',
'10.0 dias': 'Suckling',
'31.0 semanas': 'Finishing',
'23.0 semanas': 'Finishing',
'26.0 dias': 'Weaning',
'4.0 dias': 'Suckling',
'100.0 dias': 'Finishing',
'30.0 kg': 'Finishing',
'60.0 kg': 'Finishing',
'24.0 dias': 'Weaning',
'24.0 semanas': 'Finishing',
'14.0 dias': 'Suckling',
'22.0 semanas': 'Finishing',
'45.0 kg': 'Finishing',
'3.0 gr. partos': 'Sow',
'39.0 semanas': 'Finishing',
'65.0 kg': 'Finishing',
'46.0 semanas': 'Finishing',
'47.0 semanas': 'Finishing',
'273.0 kg': 'Other',
'324.0 kg': 'Other',
'80.0 dias': 'Finishing',
'9.0 meses': 'Finishing',
'23.0 dias': 'Weaning',
'48.0 meses': 'Finishing',
'12.0 meses': 'Finishing',
'22.0 dias': 'Weaning',
'8.0 dias': 'Suckling',
'32.0 semanas': 'Finishing',
'33.0 semanas': 'Finishing',
'37.0 semanas': 'Finishing',
'36.0 semanas': 'Finishing',
'66.0 dias': 'Nursery',
'27.0 dias': 'Weaning',
'dias': 'Missing',
'12.0 dias': 'Suckling',


}

# Map the 'Age' column using the English translation mapping
df['PigStage'] = df['AnimalType&Age'].map(PigAgeMapping).fillna('Missing')

print(df['PigStage'].unique())

['Missing' 'Nursery' 'Finishing' 'Sow' 'Gilt' 'Weaning' 'Suckling' 'Boar']


In [9]:
df.head(50)

,SampleNumber,Date,Province,Pathogen,Result,Age,SampleType,BreedType,FarmType,DiagnosticTest,Country,Breed,Lab_Reference,AnimalType,AnimalType&Age,PigStage
0,1,2020-07-01,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
1,1,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
2,2,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
3,2,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
4,3,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
5,3,2020-07-01,Missing,Swine influenza virus,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
6,4,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
7,4,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
8,5,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Fluid,None,None,PCR,Spain,Meat,4,,0.0,Missing
9,5,2020-07-01,Missing,Swine influenza virus,1,0.0 None,Fluid,None,None,PCR,Spain,Meat,4,,0.0,Missing


In [10]:
# Combine the results from both mappings (Farmtype and PigStages)
#def determine_category(row):
#    if pd.notnull(row['AnimalType'])and row['AnimalType'] != 'Missing':
#        return row['AnimalType']
#    elif row['Age'] in PigAgeMapping:
#        return PigAgeMapping[row['Age']]
#    else:
#        return 'Missing'

#df['PigStage'] = df.apply(determine_category, axis=1)



#

In [11]:
print(df['Pathogen'].unique())

['Mycoplasma hyopneumoniae' 'Swine influenza virus']


In [12]:
print(df['SampleType'].unique())

['Tissue' 'Fluid' 'BAL' 'Mixed' 'Swab' 'Other']


In [13]:
df.head(50)

,SampleNumber,Date,Province,Pathogen,Result,Age,SampleType,BreedType,FarmType,DiagnosticTest,Country,Breed,Lab_Reference,AnimalType,AnimalType&Age,PigStage
0,1,2020-07-01,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
1,1,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
2,2,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
3,2,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
4,3,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
5,3,2020-07-01,Missing,Swine influenza virus,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
6,4,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
7,4,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
8,5,2020-07-01,Missing,Swine influenza virus,0,0.0 None,Fluid,None,None,PCR,Spain,Meat,4,,0.0,Missing
9,5,2020-07-01,Missing,Swine influenza virus,1,0.0 None,Fluid,None,None,PCR,Spain,Meat,4,,0.0,Missing


In [14]:
df.to_csv('output/PigLab4_1.csv', index=False)

In [15]:
df = df.drop(['AnimalType'], axis=1)
df = df.drop(['Age'], axis=1)
df = df.drop(['BreedType'], axis=1)
df = df.drop(['FarmType'], axis=1)
df = df.drop(['AnimalType&Age'], axis=1)

In [16]:
df.head(50)

,SampleNumber,Date,Province,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,1,2020-07-01,Missing,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Missing
1,1,2020-07-01,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
2,2,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
3,2,2020-07-01,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
4,3,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
5,3,2020-07-01,Missing,Swine influenza virus,1,Tissue,PCR,Spain,Meat,4,Missing
6,4,2020-07-01,Missing,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing
7,4,2020-07-01,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
8,5,2020-07-01,Missing,Swine influenza virus,0,Fluid,PCR,Spain,Meat,4,Missing
9,5,2020-07-01,Missing,Swine influenza virus,1,Fluid,PCR,Spain,Meat,4,Missing


## Second data upload

In [17]:
# Define the file paths
files1 = ['Lab4/202503-Dades projecte DECIDE_carrega2_Venviada1.csv',
        'Lab4/202503-Dades projecte DECIDE_carrega2_Venviada2.csv']

# Load the data into Pandas dataframes
dfs1 = []
for file in files1:
    df1 = pd.read_csv(file, sep=";")
    dfs1.append(df1)


df1 = pd.concat(dfs1, ignore_index=True)

In [18]:
df1.head(50)

,Identificador,Fecha Registro Informe,Provincia,Prueba,Análisis,Positividadresultado,ResultadoFinalRes,Edad,TipoEdad,Material,Animal,Categoria
0,184311,03/01/2023,NaN,M. hyopneumoniae DUPLEX,M. hyopneumoniae DUPLEX,NEG,NEG,0.0,NaN,T-Pulmón,NaN,NaN
1,184312,03/01/2023,Girona,IP,IP,NEG,NEG,0.0,NaN,T-Pulmón,NaN,NaN
2,184329,04/01/2023,Barcelona,IP,IP,NEG,NEG,0.0,NaN,T-Pulmón,NaN,NaN
3,184336,04/01/2023,NaN,IP,IP,POS,"POS ++ (Ct 32,9)",0.0,NaN,T-Pulmón,Cerdos,NaN
4,184337,04/01/2023,Lleida,IP,IP,NEG,NEG,0.0,NaN,T-Pulmón,Cerdos,NaN
5,184338,04/01/2023,NaN,M. hyopneumoniae DUPLEX,M. hyopneumoniae DUPLEX,NEG,NEG,0.0,NaN,T-Pulmón,Cerdos,NaN
6,184338,04/01/2023,NaN,IP,IP,NEG,NEG,0.0,NaN,T-Pulmón,Cerdos,NaN
7,184339,04/01/2023,NaN,M. hyopneumoniae DUPLEX,M. hyopneumoniae DUPLEX,NEG,NEG,0.0,NaN,T-Pulmón,Cerdos,NaN
8,184339,04/01/2023,NaN,IP,IP,NEG,NEG,0.0,NaN,T-Pulmón,Cerdos,NaN
9,184340,04/01/2023,NaN,IP,IP,POS,"POS ++ (Ct 30,8)",0.0,NaN,T-Pulmón,Cerdos,NaN


In [19]:
# Errors arised with the coding of the Age 
import unicodedata

def clean_text(text):
    if pd.isna(text):
        return None
    text = str(text).strip().lower()
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    return text


# Create a column with the clean text
df1['Edad'] = df1['Edad'].apply(clean_text)
df1['TipoEdad'] = df1['TipoEdad'].apply(clean_text)
df1['Animal'] = df1['Animal'].apply(clean_text)
df1['Categoria'] = df1['Categoria'].apply(clean_text)
df1['Provincia'] = df1['Provincia'].apply(clean_text)

print(df1['Provincia'].unique())

[None 'girona' 'barcelona' 'lleida' 'murcia' 'segovia' 'valencia'
 'granada' 'sevilla' 'malaga' 'huesca' 'zaragoza' 'almeria' 'navarra'
 'toledo' 'albacete' 'tarragona' 'jaen' 'zamora' 'badajoz' 'la coruna'
 'burgos' 'la rioja' 'pontevedra' 'lugo' 'orense' 'cuenca' 'teruel' 'leon'
 'castellon' 'cadiz' 'salamanca' 'caceres' 'alicante' 'soria' 'cordoba'
 'huelva' 'avila' 'valladolid' 'madrid' 'alava' 'palencia' 'baleares']


In [20]:
print(df1['Prueba'].unique())

['M. hyopneumoniae DUPLEX' 'IP' 'PRRS' 'Brachyspira REAL' 'App REAL'
 'PEDV' 'PRRS FAST THERMO' 'Enterocheck PLUS' 'Enterocheck VIRUS']


In [21]:
print(df1['TipoEdad'].unique())

[None 'meses' '' 'semanas' 'dias' 'kg' 'gr. partos' 'partos']


In [22]:
print(df1['Positividadresultado'].unique())

['NEG' 'POS' 'FS']


In [23]:
#Here Age, Ageunit and Farmtype are PigStage


df1 = df1.rename(columns={
    'Identificador': 'SampleNumber',
    'Fecha Registro Informe': 'Date',
    'Provincia': 'Province',
    'Prueba': 'Pathogen',
    'Análisis': 'Pathogen1',
    'Positividadresultado': 'Result',
    'ResultadoFinalRes':'Result1',
    'Edad': 'Age', #Age of animal
    'TipoEdad': 'AgeUnit', #represents the unit used to measure the age of animals (e.g, Weeks, Month,year)
    'Material': 'SampleType',
    'Animal': 'BreedType', 
    'Categoria':'FarmType',#refers to the productive stage (or the farming period) of the animals (fattening, nursery, weaned pigs, etc.)
})

df1['DiagnosticTest'] = 'PCR'
df1['Country'] = 'Spain'
df1['Breed'] = 'Meat'
df1["Lab_Reference"] = "4"




# Combine 'Age' and 'Age Unit' columns into a new column 'Combined_Age'
df1['Age'] = df1['Age'].replace(0.0, None)
#df1['AgeUnit'] = df1['AgeUnit'].fillna('Missing')

df1['Age'] = df1['Age'].astype(str) + ' ' + df1['AgeUnit'].astype(str)

# Replace 'Missing' if both 'Age' and 'AgeUnit' are missing
#df1['Age'] = df1.apply(lambda row: 'Missing' if row['Age'] == None else row['Age'], axis=1).fillna(None)
# Replace to None if both 'Age' and 'AgeUnit' are missing
df1['Age'] = df1['Age'].apply(lambda x: None if x == 'Missing' else x)

# Drop the original 'Edad' and 'Tipo Edad' columns if needed
df1 = df1.drop(['AgeUnit'], axis=1)


In [24]:
#Annotation Mapping

#Pathogen and Sample Mapping and Annotations
PathogenMapping = {
    "M. hyopneumoniae": "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae antistoffen" : "Mycoplasma hyopneumoniae",
    "Mycoplasma hyopneumoniae" : "Mycoplasma hyopneumoniae",
    "M. hyopneumoniae DUPLEX": "Mycoplasma hyopneumoniae",
    "SIV" : "Swine influenza virus",
    "Influenza" : "Swine influenza virus",
    "IP": "Swine influenza virus",
    "App REAL": "Actinobacillus pleuropneumoniae",
    "Brachyspira REAL": "Brachyspira spp",
    "Enterocheck PLUS": "Porcine epidemic diarrhea virus",
    "Enterocheck VIRUS" : "Porcine epidemic diarrhea virus",
    "PEDV": "Porcine epidemic diarrhea virus",
    "PRRS" : "Porcine reproductive and respiratory syndrome virus",
    "PRRS FAST THERMO": "Porcine reproductive and respiratory syndrome virus"
}

df1["Pathogen"] = df1["Pathogen"].map(PathogenMapping).fillna("Missing")

SampleTypeMapping = {
    'T-Pulmón': 'Tissue',
    'F-Fluido Oral': 'Fluid',
    'F-Líquido traqueobronquial': 'BAL',
    'T-Pulmón y bronquio': 'Mixed',
    'T-Pulmón y linfonodo': 'Mixed',
    'T-Pulmón y tonsila': 'Mixed',
    'Escobillones': 'Swab',
    'Tejidos': 'Mixed',
    'T-Pulmón y timo': 'Mixed',
    'Fluídos del procesado': 'Fluid',
    'F-Líquido traqueal': 'BAL',
    'T-Lenguas': 'Tissue',
    'E-Nasales': 'Swab',
    'T-Pulmón, linfonodo y tonsila': 'Mixed',
    'T-Testículos': 'Tissue',
    'T-Pulmón y corazón': 'Mixed',
    'E-Pulmón': 'Swab',
    'T-Pulmón,hígado,riñón,bazo,int': 'Mixed',
    'FTA': 'Other',
    'T-Pulmón e hígado': 'Mixed',
    'E-Vaginales': 'Swab',
    'E-Escobillón de saliva': 'Swab',
    'T-Linfonodo': 'Tissue',
    'Fluídos corp.': 'Fluid',
    'E-Escobillón abdominal': 'Swab',
    'T-Pulmón y bazo': 'Mixed',
    'Sangre': 'Blood',
    'T-Cordón umbilical': 'Tissue',
    'Heces': 'Faeces',
    'F-Líquidos de procesado': 'Fluid',
    'T-Colas': 'Tissue',
    'T-Testículos y colas': 'Mixed',
    'T-Intestino': 'Tissue',
    'T-Pulmón, timo y linfonodo': 'Mixed',
    'F-Exudados': 'Fluid',
    'T-Pulmón, hígado y tonsila': 'Mixed',
    'T-Pulmón y cordón umbilical': 'Mixed',
    'Placa sembrada': 'Other',
    'Cepas': 'Other',
    'F-Semen': 'Fluid',
    'Vacuna': 'Other',
    'Líquido Preservante': 'Other',
    'E-Rectales': 'Swab',
    'F-Fluídos del procesado': 'Fluid',
    'RNA': 'Other',
    'Inóculo': 'Other',
    'F-Líquido pericárdico': 'Fluid',
    'F-Líquido torácico': 'Fluid',
    'T-Vísceras': 'Mixed',
    'FTA ELUTE': 'Other',
    'T-Rabo': 'Tissue',
    'Contenido intestinal': 'Faeces',
    'F-Secreción nasal': 'Fluid',
    'T-Ovario': 'Tissue',
    'T-Placenta': 'Tissue',
    'T-Embrión': 'Tissue',
    'T-Feto': 'Tissue',
    'T-Pulmón,bazo y linfonodo': 'Mixed',
    'Suero': 'Blood',
    'E-Intestino': 'Swab',
    'T-Lenguas y colas': 'Mixed',
    'E-Bronquio': 'Swab',
    'T-Pulmón,hígado,intestin, bazo': 'Mixed',
    'E-Tráquea': 'Swab',
    'Escobillón Digestivo': 'Swab',
    'T-Lenguas y testículos': 'Mixed',
    'T-Lenguas, colas y testículos': 'Mixed',
    'F-Líquidos': 'Fluid',
    'F-Líquido amniótico': 'Fluid',
    'E-Cavidad torácica': 'Swab',
    'T-Feto y placenta': 'Mixed',
    'FTA Digestivo': 'Other',
    ' ': 'Missing',

}

# Ex

# Map the SampleType column using the defined mapping
df1["SampleType"] = df1["SampleType"].map(SampleTypeMapping).fillna("Missing")


# Add Result mapping for NEG = 0 and POS =1
resultMapping = {
    "NEG": "0",
    "POS": "1",
    "FS": "Missing",
}
df1["Result"] = df1["Result"].map(resultMapping).fillna("Missing")


# Province annotation  mapping
ProvinceMapping = {
    'alava': 'Alava',
    'albacete': 'Albacete',
    'alicante': 'Alicante',
    'almeria': 'Almeria',
    'avila': 'Avila',
    'badajoz': 'Badajoz',
    'baleares': 'IslasBaleares',
    'barcelona': 'Barcelona',
    'burgos': 'Burgos',
    'caceres': 'Caceres',
    'cadiz': 'Cadiz',
    'castellon': 'Castellon',
    'cordoba': 'Cordoba',
    'cuenca': 'Cuenca',
    'girona': 'Girona',
    'granada': 'Granada',
    'huelva': 'Huelva',
    'huesca': 'Huesca',
    'jaen': 'Jaen',
    'la coruna': 'ACoruna',
    'la rioja': 'LaRioja',
    'leon': 'Leon',
    'lleida': 'Lleida',
    'lugo': 'Lugo',
    'madrid': 'Madrid',
    'malaga': 'Malaga',
    'murcia': 'Murcia',
    'navarra': 'Navarre',
    'None': 'Missing',
    'orense': 'Ourense',
    'palencia': 'Palencia',
    'pontevedra': 'Pontevedra',
    'salamanca': 'Salamanca',
    'segovia': 'Segovia',
    'sevilla': 'Sevilla',
    'soria': 'Soria',
    'tarragona': 'Tarragona',
    'teruel': 'Teruel',
    'toledo': 'Toledo',
    'valencia': 'Valencia',
    'valladolid': 'Valladolid',
    'zamora': 'Zamora',
    'zaragoza': 'Zaragoza',

}


# Translate provinces in the 'referncia' column using the mapping
df1['Province'] = df1['Province'].map(ProvinceMapping).fillna('Missing')

# Step 1: Convert 'Date' column to datetime format
df1['Date'] = pd.to_datetime(df1['Date'], format='%d/%m/%Y')

# Step 2: Format to Year, Month, Day
df1['Date'] = df1['Date'].dt.strftime("%Y-%m-%d")


In [25]:
print(df1['SampleType'].unique())

['Tissue' 'Fluid' 'Mixed' 'Swab' 'Other' 'Blood' 'Missing' 'Faeces' 'BAL']


In [26]:
print(df1['Pathogen'].unique())

['Mycoplasma hyopneumoniae' 'Swine influenza virus'
 'Porcine reproductive and respiratory syndrome virus' 'Brachyspira spp'
 'Actinobacillus pleuropneumoniae' 'Porcine epidemic diarrhea virus']


In [27]:
# Drop the original 'Pathogen1' and 'Result1' columns if needed
df1 = df1.drop(['Pathogen1', 'Result1'], axis=1)

df1.head(50)

,SampleNumber,Date,Province,Pathogen,Result,Age,SampleType,BreedType,FarmType,DiagnosticTest,Country,Breed,Lab_Reference
0,184311,2023-01-03,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4
1,184312,2023-01-03,Girona,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4
2,184329,2023-01-04,Barcelona,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4
3,184336,2023-01-04,Missing,Swine influenza virus,1,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4
4,184337,2023-01-04,Lleida,Swine influenza virus,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4
5,184338,2023-01-04,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4
6,184338,2023-01-04,Missing,Swine influenza virus,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4
7,184339,2023-01-04,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4
8,184339,2023-01-04,Missing,Swine influenza virus,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4
9,184340,2023-01-04,Missing,Swine influenza virus,1,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4


In [28]:
#df['AnimalType'] = df1[['BreedType', 'FarmType']].agg(' '.join, axis=1)
#df['AnimalType'] = df[['BreedType', 'FarmType']].astype(str).agg(' '.join, axis=1)
df1['AnimalType'] = df1[['BreedType', 'FarmType']].agg(lambda x: ' '.join(filter(None, map(str, x.dropna()))), axis=1)

#print(df['AnimalType'].unique())
#Join AnimalType & Age
#df['AnimalType&Age'] = df[['AnimalType', 'Age']].astype(str).agg(' '.join, axis=1)
df1['AnimalType&Age'] = df1[['AnimalType', 'Age']].agg(lambda x: ' '.join(filter(None, map(str, x.dropna()))), axis=1)

df1['AnimalType&Age'] = df1['AnimalType&Age'].str.replace('None', '', regex=False).str.strip()

print(df1['AnimalType&Age'].unique())

['0.0' 'cerdos 0.0' 'cerdas 0.0' 'cerdas nuliparas 7.0 meses'
 'cerdas 10.0 meses' 'cerdas gestacion 12.0 meses'
 'cerdas gestacion 14.0 meses' 'cerdas gestacion 0.0'
 'cerdas reposicion 0.0' 'cerdos engorde 0.0'
 'cerdos engorde 26.0 semanas' 'cerdos transicion 0.0' '5.0 semanas'
 '9.0 semanas' 'cerdos transicion 4.0 semanas' 'cerdos engorde 3.0 meses'
 'cerdos engorde 4.0 meses' 'cerdos engorde 6.0 meses' '6.0 semanas'
 'cerdos final transicion 10.0 semanas' 'cerdas nuliparas 0.0'
 'lechones destetados 0.0' 'cerdos 6.0 semanas'
 'cerdos transicion 10.0 semanas' 'cerdos transicion 5.0 semanas'
 'lechones 1.0 semanas' 'lechones 2.0 semanas' 'lechones 3.0 semanas'
 'lechones engorde 0.0' '7.0 semanas' '10.0 semanas'
 'cerdos final transicion 0.0' 'cerdos transicion 6.0 semanas' '2.0 meses'
 '6.0 meses' '8.0 meses' 'cerdos 4.0 semanas' 'cerdos 9.0 semanas'
 'cerdos 8.0 semanas' 'lechones 35.0 dias' 'lechones 40.0 dias'
 'cerdos destetados 0.0' 'cerdos transicion 9.0 semanas'
 'cerdos des

In [29]:
PigAgeMapping = {

   '0': 'Missing',
'cerdos final transicion 0.0': 'Nursery',
'6.0 semanas': 'Nursery',
'7.0 semanas': 'Nursery',
'11.0 semanas': 'Finishing',
'12.0 semanas': 'Finishing',
'cerdos 0.0': 'Finishing',
'cerdos 16.0 semanas': 'Finishing',
'cerdos 10.0 semanas': 'Nursery',
'cerdos transicion 10.0 semanas': 'Nursery',
'cerdos 8.0 semanas': 'Nursery',
'cerdos engorde 0.0': 'Finishing',
'10.0 semanas': 'Nursery',
'cerdos 6.0 semanas': 'Nursery',
'cerdas primiparas 0.0': 'Sow',
'cerdas multiparas 0.0': 'Sow',
'cerdas 0.0': 'Sow',
'lechones 35.0 dias': 'Nursery',
'lechones 56.0 dias': 'Nursery',
'cerdas nuliparas 0.0': 'Gilt',
'cerdos 42.0 dias': 'Nursery',
'cerdos 60.0 dias': 'Nursery',
'lechones destetados 4.0 semanas': 'Weaning',
'lechones destetados 7.0 semanas': 'Nursery',
'17.0 semanas': 'Finishing',
'lechones 5.0 semanas': 'Nursery',
'lechones 7.0 semanas': 'Nursery',
'cerdos transicion 0.0': 'Nursery',
'lechones 0.0': 'Suckling',
'90.0 kg': 'Finishing',
'cerdos final transicion 10.0 semanas': 'Nursery',
'lechones 4.0 semanas': 'Weaning',
'cerdas fut. reproduct. 0.0': 'Gilt',
'cerdos engorde 23.0 dias': 'Finishing',
'cerdas reproductoras 0.0': 'Sow',
'cerdos 9.0 semanas': 'Nursery',
'cerdos 3.0 meses': 'Finishing',
'cerdos 4.0 meses': 'Finishing',
'cerdos 7.0 meses': 'Finishing',
'cerdos 30.0 kg': 'Finishing',
'cerdos 50.0 kg': 'Finishing',
'cerdos 100.0 kg': 'Finishing',
'cerdos transicion 5.0 semanas': 'Nursery',
'cerdos engorde 11.0 semanas': 'Finishing',
'cerdos final transicion 8.0 semanas': 'Nursery',
'cerdos final transicion 60.0 dias': 'Nursery',
'cerdos final transicion 30.0 dias': 'Nursery',
'4.0 semanas': 'Weaning',
'cerdos transicion 8.0 semanas': 'Nursery',
'3.0 semanas': 'Weaning',
'cerdos 40.0 kg': 'Finishing',
'cerdos engorde 17.0 semanas': 'Finishing',
'lechones 25.0 dias': 'Weaning',
'cerdos engorde 90.0 kg': 'Finishing',
'cerdos engorde 80.0 kg': 'Finishing',
'lechones 21.0 dias': 'Weaning',
'cerdos destetados 49.0 dias': 'Nursery',
'cerdos engorde 4.0 meses': 'Finishing',
'cerdos transicion 6.0 semanas': 'Nursery',
'4.0 meses': 'Finishing',
'5.0 meses': 'Finishing',
'9.0 semanas': 'Nursery',
'cerdos 5.0 semanas': 'Nursery',
'21.0 dias': 'Weaning',
'45.0 dias': 'Nursery',
'cerdos final transicion': 'Nursery',
'lechones 11.0 semanas': 'Nursery',
'lechones destetados 0.0': 'Weaning',
'25.0 dias': 'Weaning',
'cerdos 31.0 dias': 'Nursery',
'cerdas gestacion 0.0': 'Sow',
'16.0 semanas': 'Finishing',
'lechones 9.0 semanas': 'Nursery',
'lechones lactacion 3.0 semanas': 'Suckling',
'cerdos destetados 9.0 semanas': 'Nursery',
'20.0 kg': 'Finishing',
'40.0 kg': 'Finishing',
'100.0 kg': 'Finishing',
'5.0 semanas': 'Nursery',
'cerdos 13.0 semanas': 'Finishing',
'50.0 dias': 'Nursery',
'70.0 dias': 'Nursery',
'90.0 dias': 'Finishing',
'110.0 dias': 'Finishing',
'cerdas reposicion 0.0': 'Gilt',
'verracos 0.0': 'Boar',
'8.0 semanas': 'Nursery',
'17.0 kg': 'Nursery',
'21.0 kg': 'Finishing',
'26.0 kg': 'Finishing',
'35.0 kg': 'Finishing',
'lechones lactacion 6.0 semanas': 'Nursery',
'lechones lactacion 9.0 semanas': 'Nursery',
'cerdos 7.0 semanas': 'Nursery',
'cerdos destetados 0.0': 'Weaning',
'cerdos 4.0 semanas': 'Weaning',
'cerdos 11.0 semanas': 'Finishing',
'cerdos engorde 12.0 semanas': 'Finishing',
'cerdos engorde 6.0 semanas': 'Finishing',
'cerdos engorde 9.0 semanas': 'Finishing',
'lechones 6.0 semanas': 'Nursery',
'cerdos 15.0 semanas': 'Finishing',
'cerdos 24.0 semanas': 'Finishing',
'3.0 meses': 'Finishing',
'cerdos 40.0 dias': 'Sow',
'cerdos 67.0 dias': 'Nursery',
'cerdos 28.0 dias': 'Nursery',
'lechones destetados 9.0 semanas': 'Weaning',
'cerdos engorde 3.0 meses': 'Finishing',
'cerdos 33.0 dias': 'Nursery',
'cerdos 47.0 dias': 'Nursery',
'cerdos 18.0 kg': 'Nursery',
'cerdos engorde 16.0 semanas': 'Finishing',
'cerdos 41.0 dias': 'Nursery',
'cerdos 62.0 dias': 'Nursery',
'cerdas nuliparas 4.0 meses': 'Gilt',
'cerdas nuliparas 7.0 meses': 'Gilt',
'lechones 10.0 semanas': 'Nursery',
'175.0 dias': 'Finishing',
'135.0 dias': 'Finishing',
'cerdos 2.0 meses': 'Nursery',
'lechones 23.0 dias': 'Weaning',
'lechones 58.0 dias': 'Nursery',
'6.0 meses': 'Finishing',
'18.0 semanas': 'Finishing',
'cerdas 30.0 semanas': 'Gilt',
'cerdas jovenes 0.0': 'Gilt',
'10.0 meses': 'Finishing',
'cerdas 5.0 meses': 'Gilt',
'cerdos transicion 9.0 semanas': 'Nursery',
'cerdos engorde 14.0 semanas': 'Finishing',
'cerdos engorde 10.0 semanas': 'Finishing',
'cerdos 18.0 semanas': 'Finishing',
'8.0 kg': 'Nursery',
'12.0 kg': 'Nursery',
'18.0 kg': 'Nursery',
'110.0 kg': 'Finishing',
'cerdos engorde 50.0 kg': 'Finishing',
'cerdos engorde 70.0 kg': 'Finishing',
'30.0 dias': 'Nursery',
'40.0 dias': 'Nursery',
'lechones 8.0 semanas': 'Nursery',
'cerdas reposicion 5.0 meses': 'Gilt',
'60.0 dias': 'Nursery',
'lechones 3.0 semanas': 'Weaning',
'cerdos destetados 7.0 semanas': 'Nursery',
'lechones 12.0 semanas': 'Finishing',
'cerdos 12.0 semanas': 'Finishing',
'cerdos 14.0 semanas': 'Finishing',
'63.0 dias': 'Nursery',
'49.0 dias': 'Nursery',
'15.0 semanas': 'Finishing',
'13.0 semanas': 'Finishing',
'cerdas fut. reproduct. 4.0 meses': 'Gilt',
'cerdas 4.0 semanas': 'Gilt',
'cerdas 6.0 semanas': 'Gilt',
'cerdas 9.0 semanas': 'Gilt',
'cerdos transicion 2.0 semanas': 'Nursery',
'cerdos transicion 1.0 semanas': 'Nursery',
'cerdos transicion 4.0 semanas': 'Nursery',
'cerdos transicion 7.0 semanas': 'Nursery',
'cerdas reposicion 6.0 meses': 'Gilt',
'cerdos engorde 15.0 semanas': 'Finishing',
'42.0 dias': 'Nursery',
'cerdos engorde 25.0 semanas': 'Finishing',
'cerdos 20.0 semanas': 'Finishing',
'cerdos 3.0 semanas': 'Weaning',
'cerdos engorde 18.0 semanas': 'Finishing',
'cerdos 105.0 dias': 'Finishing',
'cerdos 90.0 dias': 'Finishing',
'cerdos 75.0 dias': 'Finishing',
'cerdos 50.0 dias': 'Nursery',
'lechones 45.0 dias': 'Nursery',
'cerdos transicion 3.0 semanas': 'Nursery',
'lechones destetados 32.0 dias': 'Nursery',
'cerdos engorde 90.0 dias': 'Finishing',
'cerdas reposicion 8.0 meses': 'Gilt',
'cerdos engorde 40.0 kg': 'Finishing',
'cerdos 80.0 kg': 'Finishing',
'cerdos transicion 12.0 semanas': 'Nursery',
'cerdos transicion 30.0 dias': 'Nursery',
'cerdos transicion 60.0 dias': 'Nursery',
'cerdos transicion 45.0 dias': 'Nursery',
'lechones destetados 10.0 semanas': 'Nursery',
'cerdos engorde 13.0 semanas': 'Finishing',
'lechones 37.0 dias': 'Nursery',
'lechones 31.0 dias': 'Nursery',
'lechones 52.0 dias': 'Nursery',
'cerdos 35.0 dias': 'Nursery',
'cerdos engorde 27.0 semanas': 'Finishing',
'cerdos engorde 26.0 semanas': 'Finishing',
'cerdos 25.0 semanas': 'Finishing',
'cerdos 27.0 semanas': 'Finishing',
'cerdos engorde 28.0 semanas': 'Finishing',
'cerdas jovenes 5.0 meses': 'Gilt',
'cerdos engorde 100.0 kg': 'Finishing',
'26.0 semanas': 'Finishing',
'27.0 semanas': 'Finishing',
'25.0 semanas': 'Finishing',
'final transicion 0.0': 'Nursery',
'50.0 kg': 'Finishing',
'cerdos 5.0 meses': 'Finishing',
'cerdos transicion 42.0 dias': 'Nursery',
'cerdos engorde 23.0 semanas': 'Finishing',
'cerdos engorde 30.0 kg': 'Finishing',
'cerdos engorde 60.0 kg': 'Finishing',
'cerdos engorde 7.0 meses': 'Finishing',
'lechones destetados 5.0 semanas': 'Nursery',
'32.0 dias': 'Nursery',
'54.0 dias': 'Nursery',
'lechones 32.0 dias': 'Nursery',
'lechones 63.0 dias': 'Nursery',
'lechones 41.0 dias': 'Nursery',
'cerdos destetados 60.0 kg': 'Finishing',
'cerdos 140.0 dias': 'Finishing',
'cerdos 130.0 dias': 'Finishing',
'cerdas 10.0 meses': 'Gilt',
'cerdas gestacion 12.0 meses': 'Sow',
'cerdas gestacion 14.0 meses': 'Sow',
'cerdos engorde 6.0 meses': 'Finishing',
'lechones 1.0 semanas': 'Suckling',
'lechones 2.0 semanas': 'Suckling',
'lechones engorde 0.0': 'Finishing',
'2.0 meses': 'Nursery',
'8.0 meses': 'Finishing',
'lechones 40.0 dias': 'Nursery',
'cerdos destetados 8.0 semanas': 'Nursery',
'lechones 20.0 kg': 'Finishing',
'lechones lactacion 1.0 semanas': 'Suckling',
'cerdos final transicion 9.0 semanas': 'Nursery',
'cerdos engorde 2.0 meses': 'Finishing',
'cerdos engorde': 'Finishing',
'lechones destetados': 'Weaning',
'cerdos 16.0 kg': 'Nursery',
'cerdos 10.0 meses': 'Finishing',
'cerdos 49.0 dias': 'Nursery',
'lechones lactacion 0.0': 'Suckling',
'56.0 dias': 'Nursery',
'cerdos engorde 25.0 kg': 'Finishing',
'cerdos 10.0 kg': 'Nursery',
'cerdas reposicion 7.0 meses': 'Gilt',
'15.0 dias': 'Suckling',
'35.0 dias': 'Nursery',
'lechones 28.0 dias': 'Weaning',
'lechones destetados 6.0 semanas': 'Nursery',
'cerdos transicion 59.0 dias': 'Nursery',
'cerdos transicion 38.0 dias': 'Nursery',
'cerdos': 'Finishing',
'lechones 70.0 dias': 'Nursery',
'cerdos final transicion 70.0 dias': 'Nursery',
'lechones destetados 8.0 semanas': 'Nursery',
'14.0 semanas': 'Finishing',
'lechones destetados 42.0 dias': 'Nursery',
'lechones destetados 60.0 dias': 'Nursery',
'cerdos 54.0 dias': 'Nursery',
'lechones 48.0 dias': 'Nursery',
'lechones 74.0 dias': 'Nursery',
'cerdos transicion 50.0 dias': 'Nursery',
'lechones destetados 45.0 dias': 'Nursery',
'1.0 semanas': 'Suckling',
'lechones 1.0 dias': 'Suckling',
'lechones 15.0 dias': 'Suckling',
'cerdos transicion 21.0 dias': 'Weaning',
'cerdos transicion 35.0 dias': 'Nursery',
'cerdos transicion 53.0 dias': 'Nursery',
'cerdos transicion 25.0 dias': 'Weaning',
'cerdos 45.0 dias': 'Nursery',
'cerdos 70.0 dias': 'Nursery',
'75.0 dias': 'Finishing',
'65.0 dias': 'Nursery',
'55.0 dias': 'Nursery',
'cerdas 24.0 semanas': 'Finishing',
'cerdas 18.0 semanas': 'Finishing',
'cerdas 15.0 semanas': 'Finishing',
'lechones 10.0 dias': 'Suckling',
'cerdos transicion 11.0 semanas': 'Finishing',
'cerdos engorde 5.0 semanas': 'Nursery',
'cerdos engorde 7.0 semanas': 'Nursery',
'lechones 54.0 dias': 'Nursery',
'cerdos 30.0 semanas': 'Finishing',
'cerdos 30.0 dias': 'Nursery',
'cerdos 55.0 dias': 'Nursery',
'lechones 17.0 kg': 'Nursery',
'cerdos 25.0 dias': 'Weaning',
'cerdos engorde 4.0 semanas': 'Weaning',
'28.0 dias': 'Weaning',
'cerdos 29.0 semanas': 'Finishing',
'cerdos 85.0 dias': 'Nursery',
'cerdos 65.0 dias': 'Nursery',
'cerdos engorde 5.0 meses': 'Finishing',
'lechones 20.0 dias': 'Suckling',
'cerdos final transicion 12.0 semanas': 'Finishing',
'28.0 semanas': 'Finishing',
'lechones destetados 3.0 semanas': 'Weaning',
'cerdas primiparas 6.0 meses': 'Gilt',
'cerdos 100.0 dias': 'Finishing',
'cerdos destetados 40.0 dias': 'Nursery',
'cerdos destetados 60.0 dias': 'Nursery',
'52.0 dias': 'Nursery',
'cerdos engorde 20.0 semanas': 'Finishing',
'21.0 semanas': 'Finishing',
'19.0 semanas': 'Finishing',
'lechones destetados 32.0 semanas': 'Nursery',
'cerdos 6.0 meses': 'Finishing',
'lechones destetados 38.0 dias': 'Nursery',
'lechones destetados 67.0 dias': 'Nursery',
'34.0 semanas': 'Finishing',
'cerdos engorde 22.0 semanas': 'Finishing',
'lechones 17.0 dias': 'Suckling',
'lechones 14.0 dias': 'Suckling',
'lechones 11.0 dias': 'Suckling',
'cerdos engorde 21.0 semanas': 'Finishing',
'7.0 meses': 'Finishing',
'lechones 60.0 dias': 'Nursery',
'cerdos 1.0 semanas': 'Suckling',
'cerdos transicion 70.0 dias': 'Nursery',
'cerdos 1.0 meses': 'Weaning',
'lechones nuliparas 60.0 dias': 'Other',
'lechones nacido debil 0.0': 'Suckling',
'lechones 18.0 dias': 'Suckling',
'cerdas nuliparas 1.0 semanas': 'Gilt',
'cerdas nuliparas 40.0 kg': 'Gilt',
'cerdas nuliparas 6.0 meses': 'Gilt',
'cerdas final transicion 0.0': 'Nursery',
'cerdos final transicion 5.0 semanas': 'Nursery',
'cerdos final transicion 7.0 semanas': 'Nursery',
'cerdos final transicion 6.0 semanas': 'Nursery',
'cerdas reproductoras 9.0 semanas': 'Gilt',
'lechones 2.0': 'Suckling',
'cerdos engorde 8.0 semanas': 'Nursery',
'lechones 4.0 dias': 'Suckling',
'verracos 5.0 semanas': 'Boar',
'lechones 3.0 dias': 'Suckling',
'cerdas nuliparas 2.0 meses': 'Gilt',
'cerdas nuliparas 3.0 meses': 'Gilt',
'cerdas nuliparas 90.0 dias': 'Gilt',
'cerdas nuliparas 120.0 dias': 'Gilt',
'cerdas nuliparas 150.0 dias': 'Gilt',
'cerdos final transicion 4.0 semanas': 'Nursery',
'cerdas 7.0 meses': 'Gilt',
'lechones': 'Suckling',
'cerdos transicion 49.0 dias': 'Nursery',
'cerdas 8.0 meses': 'Gilt',
'cerdos 60.0 kg': 'Finishing',
'cerdas nuliparas 8.0 meses': 'Gilt',
'cerdas 3.0 semanas': 'Gilt',
'cerdas 5.0 semanas': 'Gilt',
'cerdos engorde 32.0 semanas': 'Finishing',
'lechones lactantes 0.0': 'Suckling',
'cerdas reposicion 60.0 kg': 'Gilt',
'cerdas reposicion 40.0 kg': 'Gilt',
'cerdas reposicion 180.0 kg': 'Gilt',
'cerdos transicion': 'Nursery',
'cerdas 1.0 gr. partos': 'Sow',
'cerdas nuliparas 13.0 semanas': 'Gilt',
'cerdas nuliparas 22.0 semanas': 'Gilt',
'cerdas nuliparas 27.0 semanas': 'Gilt',
'lechones lactantes 21.0 dias': 'Suckling',
'lechones lactantes 3.0 semanas': 'Weaning',
'lechones 27.0 dias': 'Weaning',
'lechones lactantes 24.0 dias': 'Suckling',
'lechones 24.0 dias': 'Weaning',
'cerdas nuliparas 180.0 dias': 'Gilt',
'cerdas nuliparas 220.0 dias': 'Gilt',
'cerdas nuliparas 200.0 dias': 'Gilt',
'lechones lactantes 18.0 dias': 'Suckling',
'cerdas nuliparas 160.0 dias': 'Gilt',
'cerdos final transicion 11.0 semanas': 'Finishing',
'cerdas transicion 6.0 semanas': 'Nursery',
'cerdas 24.0 meses': 'Sow',
'lechones transicion 0.0': 'Nursery',
'cerdos final transicion 75.0 dias': 'Finishing',
'lechones 5.0 dias': 'Suckling',
'lechones 2.0 dias': 'Suckling',
'lechones destetados 21.0 dias': 'Weaning',
'lechones 34.0 dias': 'Nursery',
'lechones 50.0 dias': 'Nursery',
'cerdos transicion 2.0 meses': 'Nursery',
'lechones destetados 18.0 dias': 'Suckling',
'cerdas reposicion 9.0 meses': 'Gilt',
'lechones 30.0 dias': 'Nursery',
'lechones lactacion 21.0 dias': 'Weaning',
'lechones lactacion 10.0 dias': 'Suckling',
'cerdos transicion 65.0 dias': 'Nursery',
'cerdas nuliparas 4.0 semanas': 'Gilt',
'cerdas nuliparas 6.0 semanas': 'Gilt',
'lechones 7.0 dias': 'Suckling',
'cerdos 17.0 semanas': 'Finishing',
'lechones destetados 2.0 semanas': 'Weaning',
'cerdas 8.0 semanas': 'Gilt',
'cerdos engorde 30.0 semanas': 'Finishing',
'cerdas lactantes 0.0': 'Sow',
'lechones 26.0 dias': 'Weaning',
'cerdas reproductoras 24.0 meses': 'Sow',
'cerdos 14.0 dias': 'Suckling',
'lechones nacido debil 2.0 semanas': 'Suckling',
'lechones nacido debil 3.0 semanas': 'Weaning',
'cerdos engorde 24.0 semanas': 'Finishing',
'cerdas nuliparas 8.0 semanas': 'Gilt',
'cerdos 21.0 dias': 'Weaning',
'cerdas 7.0 semanas': 'Gilt',
'lechones destetados 2.0 dias': 'Suckling',
'cerdos 15.0 kg': 'Nursery',
'verracos nuliparas 0.0': 'Other',
'lechones 2.0 meses': 'Nursery',
'cerdas nuliparas': 'Gilt',
'cerdos destetados 10.0 semanas': 'Nursery',
'lechones 49.0 dias': 'Nursery',
'cerdos 10.0 dias': 'Suckling',
'cerdas reproductoras 40.0 kg': 'Gilt',
'cerdas nuliparas 7.0 semanas': 'Gilt',
'cerdas nuliparas 5.0 semanas': 'Gilt',
'cerdos 19.0 semanas': 'Finishing',
'cerdos transicion 16.0 semanas': 'Finishing',
'lechones final transicion 12.0 semanas': 'Finishing',
'cerdos final transicion 90.0 kg': 'Finishing',
'cerdos 20.0 kg': 'Finishing',
'cerdos 80.0 dias': 'Finishing',
'cerdas  semanas': 'Sow',
'cerdas nuliparas 28.0 dias': 'Gilt',
'lechones 0.0 semanas': 'Suckling',
'lechones 3.0 meses': 'Weaning',
'lechones 0.0 dias': 'Suckling',
'cerdas reposicion 100.0 kg': 'Gilt',
'cerdas reposicion 80.0 kg': 'Gilt',
'cerdos destetados 5.0 semanas': 'Nursery',
'cerdos 38.0 dias': 'Nursery',
'cerdas nuliparas 100.0 kg': 'Gilt',
'lechones destetados 28.0 dias': 'Weaning',
'lechones destetados 24.0 dias': 'Weaning',
'cerdos transicion 10.0 dias': 'Suckling',
'cerdos transicion 17.0 dias': 'Suckling',
'cerdos transicion 24.0 dias': 'Weaning',
'cerdas fut. reproduct. 8.0 meses': 'Gilt',
'cerdos engorde 7.0 dias': 'Suckling',
'cerdos final transicion 3.0 semanas': 'Nursery',
'cerdas primiparas 11.0 meses': 'Sow',
'lechones 21.0 kg': 'Finishing',
'lechones lactacion 4.0 semanas': 'Weaning',
'cerdas 12.0 meses': 'Sow',
'cerdos 22.0 semanas': 'Finishing',
'cerdos engorde 150.0 dias': 'Finishing',
'cerdos transicion 24.0 semanas': 'Finishing',
'lechones 19.0 semanas': 'Finishing',
'lechones 22.0 semanas': 'Finishing',
'cerdos 2.0 semanas': 'Suckling',
'cerdos 21.0 semanas': 'Finishing',
'cerdos destetados 28.0 dias': 'Weaning',
'cerdos 43.0 dias': 'Nursery',
'cerdos 46.0 dias': 'Nursery',
'cerdos 2.0 dias': 'Suckling',
'cerdos 6.0': 'Other',
'lechones 90.0 dias': 'Finishing',
'lechones 110.0 dias': 'Finishing',
'lechones lactacion 15.0 dias': 'Suckling',
'cerdas reposicion 28.0 dias': 'Gilt',
'cerdas': 'Sow',
'cerdas 2.0 partos': 'Sow',
'cerdos 0.0 dias': 'Other',
'cerdos transicion 28.0 dias': 'Weaning',
'cerdos transicion 41.0 dias': 'Nursery',
'cerdos transicion 63.0 dias': 'Nursery',
'cerdos 200.0 dias': 'Finishing',
'lechones lactacion 7.0 dias': 'Suckling',
'lechones lactacion 22.0 dias': 'Weaning',
'lechones 22.0 dias': 'Weaning',
'cerdos transicion 7.0 dias': 'Suckling',
'cerdas 2.0 gr. partos': 'Sow',
'cerdas 4.0 gr. partos': 'Sow',
'cerdas 6.0 gr. partos': 'Sow',
'lechones 15.0 kg': 'Nursery',
'lechones 25.0 kg': 'Finishing',
'cerdas nuliparas 0.0 partos': 'Gilt',
'cerdas 5.0 gr. partos': 'Sow',
'lechones 14.0 semanas': 'Finishing',
'cerdas 18.0 meses': 'Sow',
'cerdas 11.0 semanas': 'Gilt',
'verracos 180.0 dias': 'Boar',
'cerdas 180.0 dias': 'Gilt',
'cerdos engorde 15.0': 'Finishing',
'cerdos transicion 54.0 dias': 'Nursery',
'cerdas reposicion 170.0 dias': 'Gilt',
'lechones 1.0 partos': 'Other',
'lechones 2.0 partos': 'Other',
'lechones 3.0 partos': 'Other',
'lechones 4.0 partos': 'Other',
'lechones 5.0 partos': 'Other',
'cerdas 4.0 meses': 'Gilt',
'lechones nuliparas 0.0': 'Other',
'cerdas 1.0 semanas': 'Gilt',
'cerdas 48.0 meses': 'Sow',
'lechones destetados 15.0 dias': 'Suckling',
'lechones destetados 30.0 dias': 'Nursery',
'cerdos 3.0 dias': 'Suckling',
'lechones 9.0 dias': 'Suckling',
'cerdas multiparas 4.0 semanas': 'Sow',
'cerdas multiparas 5.0 semanas': 'Sow',
'lechones 6.0 dias': 'Suckling',
'cerdos transicion 9.0 meses': 'Finishing',
'lechones 36.0 dias': 'Nursery',
'cerdas nuliparas 3.0 semanas': 'Gilt',
'cerdas multiparas 3.0 semanas': 'Sow',
'lechones 16.0 semanas': 'Finishing',
'lechones lactacion 5.0 dias': 'Suckling',
'lechones 44.0 dias': 'Nursery',
'cerdas 28.0 semanas': 'Gilt',
'cerdos engorde 26.0 dias': 'Weaning',
'lechones 42.0 dias': 'Nursery',
'lechones 64.0 dias': 'Nursery',
'cerdos 20.0 dias': 'Nursery',
'lechones lactacion 1.0 dias': 'Suckling',
'verracos engorde 0.0': 'Boar',
'lechones 12.0 dias': 'Suckling',
'cerdos transicion 56.0 dias': 'Nursery',
'cerdos transicion 66.0 dias': 'Nursery',
'cerdas 2.0 meses': 'Gilt',
'lechones reproductoras 24.0 dias': 'Weaning',
'cerdas reproductoras 20.0 dias': 'Gilt',
'cerdas reproductoras 40.0 dias': 'Gilt',
'cerdas reproductoras 60.0 dias': 'Gilt',
'cerdos transicion 14.0 semanas': 'Finishing',
'lechones 13.0 semanas': 'Finishing',
'cerdas nuliparas 5.0 meses': 'Gilt',
'cerdos 57.0 dias': 'Nursery',
'cerdos 23.0 dias': 'Weaning',
'lechones 29.0 dias': 'Nursery',
'cerdos destetados 6.0 semanas': 'Nursery',
'verracos destetados 0.0': 'Weaning',
'lechones 8.0 dias': 'Suckling',
'cerdos 56.0 dias': 'Nursery',
'lechones 77.0 dias': 'Finishing',
'lechones 84.0 dias': 'Finishing',
'lechones  dias': 'Suckling',
'cerdos  semanas': 'Other',
'lechones destetados 11.0 semanas': 'Finishing',
'lechones 17.0': 'Suckling',
'lechones 18.0': 'Suckling',
'lechones 20.0': 'Suckling',
'lechones 1.0': 'Suckling',
'lechones lactacion 25.0 dias': 'Weaning',
'cerdos destetados 55.0 dias': 'Nursery',
'cerdos 26.0 semanas': 'Finishing',
'lechones 19.0 dias': 'Suckling',
'cerdos 120.0 dias': 'Finishing',
'cerdos 160.0 dias': 'Finishing',
'cerdos 180.0 dias': 'Finishing',
'cerdos transicion 40.0 dias': 'Nursery',
'cerdas fut. reproduct. 10.0 meses': 'Gilt',
'cerdos transicion 150.0 dias': 'Finishing',
'cerdos transicion 130.0 dias': 'Finishing',
'cerdos transicion 115.0 dias': 'Finishing',
'cerdos transicion 94.0 dias': 'Finishing',
'lechones 70.0 kg': 'Finishing',
'lechones final transicion 3.0 semanas': 'Suckling',
'lechones 8.0 meses': 'Finishing',
'cerdas 6.0 meses': 'Gilt',
'lechones lactacion 2.0 semanas': 'Suckling',
'cerdos 10.0': 'Other',
'lechones destetados 26.0 dias': 'Weaning',
'cerdos transicion 58.0 dias': 'Nursery',
'cerdos 24.0 dias': 'Weaning',
'cerdos engorde 22.0 kg': 'Finishing',
'cerdos engorde 110.0 kg': 'Finishing',
'cerdas nuliparas 28.0 semanas': 'Gilt',
'lechones nacido debil 0.0 dias': 'Suckling',
'cerdas nacido debil 0.0': 'Sow',
'lechones 75.0 dias': 'Finishing',
'lechones lactacion 2.0 dias': 'Suckling',
'cerdas primiparas 2.0 dias': 'Sow',
'cerdas multiparas 2.0 dias': 'Sow',
'lechones multiparas 0.0': 'Other',
'cerdos engorde 22.0 dias': 'Weaning',
'cerdas fut. reproduct. 50.0 kg': 'Gilt',
'verracos 4.0 meses': 'Boar',
'lechones 72.0 dias': 'Finishing',
'cerdos final transicion 4.0': 'Nursery',
'cerdos 1.0 dias': 'Suckling',
'cerdos 7.0 dias': 'Suckling',
'cerdos 23.0 semanas': 'Finishing',
'cerdas primiparas 8.0 meses': 'Sow',
'cerdos destetados 25.0 dias': 'Weaning',
'lechones destetados 8.0 dias': 'Weaning',
'cerdos multiparas 0.0': 'Sow',
'cerdos nuliparas 0.0': 'Gilt',
'lechones nacido debil 4.0 semanas': 'Weaning',
'cerdos transicion 15.0 semanas': 'Finishing',
'cerdos 98.0 dias': 'Finishing',
'cerdas reposicion 180.0 dias': 'Gilt',
'lechones 6.0 kg': 'Weaning',
'cerdas reposicion 100.0 dias': 'Gilt',
'cerdas reposicion 120.0 dias': 'Gilt',
'cerdos final transicion 50.0 dias': 'Nursery',
'cerdos 150.0 dias': 'Finishing',
'cerdos 5.0 dias': 'Suckling',
'lechones destetados 10.0 dias': 'Suckling',
'lechones nacido debil 2.0 dias': 'Suckling',
'lechones destetados 43.0 dias': 'Nursery',
'cerdas fut. reproduct. 80.0 kg': 'Gilt',
'cerdas reposicion 15.0 semanas': 'Gilt',
'cerdas reposicion 18.0 semanas': 'Gilt',
'cerdos 29.0 dias': 'Nursery',
'lechones 66.0 dias': 'Nursery',
'cerdos transicion 80.0 dias': 'Finishing',
'lechones 62.0 dias': 'Nursery',
'cerdos 61.0 dias': 'Nursery',
'lechones final transicion 0.0': 'Nursery',
'cerdos final transicion 90.0 dias': 'Finishing',
'cerdos nacido debil 0.0': 'Suckling',
'lechones 40.0 kg': 'Finishing',
'lechones primiparas 4.0 semanas': 'Other',
'cerdos transicion 71.0 dias': 'Finishing',
'lechones 16.0 dias': 'Suckling',
'cerdas 3.0 gr. partos': 'Sow',
'cerdas 7.0 gr. partos': 'Sow',
'cerdos transicion 5.0 dias': 'Suckling',
'lechones 73.0 dias': 'Finishing',
'cerdas fut. reproduct. 5.0 semanas': 'Gilt',
'cerdos engorde 19.0 semanas': 'Finishing',
'lechones destetados 1.0 semanas': 'Suckling',
'verracos': 'Boar',
'cerdos engorde 17.0 kg': 'Finishing',
'cerdos transicion 55.0 dias': 'Nursery',
'lechones 30.0 kg': 'Finishing',
'lechones lactacion 23.0 dias': 'Weaning',
'lechones 25.0 semanas': 'Finishing',
'lechones 7.0 kg': 'Weaning',
'cerdos 15.0 dias': 'Suckling',
'reproductoras 0.0': 'Sow',
'lechones 26.0 semanas': 'Finishing',
'lechones 27.0 semanas': 'Finishing',
'lechones 23.0 semanas': 'Finishing',
'lechones 24.0 semanas': 'Finishing',
'lechones 53.0 dias': 'Nursery',
'lechones nacido debil 15.0 dias': 'Suckling',
'lechones 1.0 meses': 'Weaning',
'cerdas reposicion 200.0 dias': 'Gilt',
'cerdas reposicion 187.0 dias': 'Gilt',
'cerdas reposicion 173.0 dias': 'Gilt',
'cerdas reposicion 155.0 dias': 'Gilt',
'cerdos engorde 20.0 kg': 'Finishing',
'cerdos 90.0 kg': 'Finishing',
'lechones 41.0 semanas': 'Nursery',
'lechones 42.0 semanas': 'Nursery',
'cerdas reposicion 120.0 kg': 'Gilt',
'cerdas reposicion 70.0 kg': 'Gilt',
'cerdas reproductoras': 'Sow',
'cerdos 8.0 meses': 'Finishing',
'cerdos 140.0 kg': 'Finishing',
'cerdos 135.0 kg': 'Finishing',
'cerdos 125.0 kg': 'Finishing',
'cerdos 110.0 kg': 'Finishing',
'cerdos 105.0 kg': 'Finishing',
'cerdos 93.0 kg': 'Finishing',
'cerdos 70.0 kg': 'Finishing',
'cerdos 65.0 kg': 'Finishing',
'cerdos 55.0 kg': 'Finishing',
'cerdas 25.0 dias': 'Gilt',
'cerdas reposicion 4.0 meses': 'Gilt',
'lechones  semanas': 'Suckling',
'lechones 46.0 dias': 'Nursery',
'cerdos engorde 1.0 semanas': 'Suckling',
'cerdos engorde 35.0 kg': 'Finishing',
'cerdas 21.0 dias': 'Gilt',
'lechones 8.0 kg': 'Nursery',
'lechones destetados 50.0 dias': 'Nursery',
'lechones destetados 70.0 dias': 'Nursery',
'cerdas reposicion 22.0 semanas': 'Gilt',
'cerdas 60.0 kg': 'Gilt',
'lechones 13.0 dias': 'Suckling',
'lechones 20.0 semanas': 'Finishing',
'cerdos 250.0 dias': 'Finishing',
'cerdas nuliparas 33.0 semanas': 'Gilt',
'cerdas nuliparas 20.0 semanas': 'Gilt',
'lechones 19.0 kg': 'Nursery',
'cerdas reposicion 110.0 kg': 'Gilt',
'cerdos 63.0 dias': 'Nursery',
'cerdos transicion 8.0 dias': 'Suckling',
'lechones lactacion': 'Suckling',
'lechones destetados 25.0 dias': 'Weaning',
'lechones lactacion 20.0 dias': 'Suckling',
'lechones nacido debil 1.0 dias': 'Suckling',
'cerdas 1.0 partos': 'Sow',
'cerdas 3.0 partos': 'Sow',
'cerdas 4.0 partos': 'Sow',
'cerdas reproductoras 18.0 meses': 'Sow',
'cerdos transicion 6.0': 'Nursery',
'lechones 4.0': 'Suckling',
'cerdos destetados 21.0 dias': 'Weaning',
'cerdas fut. reproduct. 12.0 semanas': 'Gilt',
'cerdas multiparas 1.0 semanas': 'Sow',
'lechones 55.0 dias': 'Nursery',
'lechones lactacion 18.0 dias': 'Suckling',
'lechones destetados 3.0 dias': 'Suckling',
'lechones destetados 52.0 dias': 'Nursery',
'lechones lactacion 3.0 dias': 'Suckling',
'lechones 33.0 semanas': 'Finishing',
'cerdas 35.0 dias': 'Gilt',
'cerdos destetados 4.0 semanas': 'Weaning',
'cerdos 27.0 dias': 'Weaning',
'cerdos destetados 80.0 dias': 'Finishing',
'cerdos engorde 31.0 semanas': 'Finishing',
'lechones 80.0 dias': 'Finishing',
'cerdos destetados 70.0 dias': 'Nursery',
'cerdas 20.0 meses': 'Gilt',
'cerdos destetados': 'Weaning',
'lechones 65.0 dias': 'Nursery',
'lechones lactacion 30.0 dias': 'Weaning',
'lechones destetados 22.0 dias': 'Weaning',
'cerdos engorde 180.0 dias': 'Finishing',
'cerdos engorde 160.0 dias': 'Finishing',
'cerdos engorde 140.0 dias': 'Finishing',
'cerdos engorde 120.0 dias': 'Finishing',
'lechones destetados 34.0 dias': 'Nursery',
'cerdas 32.0 semanas': 'Gilt',
'lechones 33.0 dias': 'Nursery',
'lechones lactacion 28.0 dias': 'Weaning',
'cerdas reposicion 160.0 dias': 'Gilt',
'cerdas reposicion 130.0 dias': 'Gilt',
'cerdas reposicion 150.0 dias': 'Gilt',
'cerdas jovenes 22.0 dias': 'Gilt',
'cerdas 22.0 dias': 'Gilt',
'cerdas primiparas 23.0 dias': 'Gilt',
'cerdas multiparas 23.0 dias': 'Gilt',
'cerdas 20.0 dias': 'Gilt',
'cerdas 42.0 dias': 'Gilt',
'cerdas 56.0 dias': 'Gilt',
'lechones 47.0 dias': 'Nursery',
'3.0 dias': 'Suckling',
'2.0 dias': 'Suckling',
'80.0 kg': 'Finishing',
'30.0 semanas': 'Finishing',
'2.0 semanas': 'Suckling',
'20.0 semanas': 'Finishing',
'103.0 dias': 'Finishing',
'3': 'Missing',
'36.0 dias': 'Nursery',
'0.0 semanas': 'Missing',
'1.0 dias': 'Suckling',
'51.0 dias': 'Nursery',
'5.0 dias': 'Suckling',
'180.0 dias': 'Finishing',
'150.0 dias': 'Finishing',
'0.0 dias': 'Missing',
'25.0 kg': 'Finishing',
'70.0 kg': 'Finishing',
'120.0 kg': 'Finishing',
'160.0 dias': 'Finishing',
'39.0 dias': 'Nursery',
'1.0 partos': 'Sow',
'77.0 dias': 'Finishing',
'7.0 dias': 'Suckling',
'20.0 dias': 'Suckling',
'170.0 dias': 'Finishing',
'200.0 dias': 'Finishing',
'140.0 dias': 'Finishing',
'120.0 dias': 'Finishing',
'6.0 dias': 'Suckling',
'10.0 dias': 'Suckling',
'31.0 semanas': 'Finishing',
'23.0 semanas': 'Finishing',
'26.0 dias': 'Weaning',
'4.0 dias': 'Suckling',
'100.0 dias': 'Finishing',
'30.0 kg': 'Finishing',
'60.0 kg': 'Finishing',
'24.0 dias': 'Weaning',
'24.0 semanas': 'Finishing',
'14.0 dias': 'Suckling',
'22.0 semanas': 'Finishing',
'45.0 kg': 'Finishing',
'3.0 gr. partos': 'Sow',
'39.0 semanas': 'Finishing',
'65.0 kg': 'Finishing',
'46.0 semanas': 'Finishing',
'47.0 semanas': 'Finishing',
'273.0 kg': 'Other',
'324.0 kg': 'Other',
'80.0 dias': 'Finishing',
'9.0 meses': 'Finishing',
'23.0 dias': 'Weaning',
'48.0 meses': 'Finishing',
'12.0 meses': 'Finishing',
'22.0 dias': 'Weaning',
'8.0 dias': 'Suckling',
'32.0 semanas': 'Finishing',
'33.0 semanas': 'Finishing',
'37.0 semanas': 'Finishing',
'36.0 semanas': 'Finishing',
'66.0 dias': 'Nursery',
'27.0 dias': 'Weaning',
'dias': 'Missing',
'12.0 dias': 'Suckling',


}
# Map the 'Age' column using the English translation mapping
df1['PigStage'] = df1['AnimalType&Age'].map(PigAgeMapping).fillna('Missing')

print(df1['PigStage'].unique())

['Missing' 'Finishing' 'Sow' 'Gilt' 'Nursery' 'Weaning' 'Suckling' 'Boar'
 'Other']


In [30]:
df1.head(50)

,SampleNumber,Date,Province,Pathogen,Result,Age,SampleType,BreedType,FarmType,DiagnosticTest,Country,Breed,Lab_Reference,AnimalType,AnimalType&Age,PigStage
0,184311,2023-01-03,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
1,184312,2023-01-03,Girona,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
2,184329,2023-01-04,Barcelona,Swine influenza virus,0,0.0 None,Tissue,None,None,PCR,Spain,Meat,4,,0.0,Missing
3,184336,2023-01-04,Missing,Swine influenza virus,1,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing
4,184337,2023-01-04,Lleida,Swine influenza virus,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing
5,184338,2023-01-04,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing
6,184338,2023-01-04,Missing,Swine influenza virus,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing
7,184339,2023-01-04,Missing,Mycoplasma hyopneumoniae,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing
8,184339,2023-01-04,Missing,Swine influenza virus,0,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing
9,184340,2023-01-04,Missing,Swine influenza virus,1,0.0 None,Tissue,cerdos,None,PCR,Spain,Meat,4,cerdos,cerdos 0.0,Finishing


In [31]:
df1.to_csv('output/PigLab4_2.csv', index=False)

In [32]:
df1 = df1.drop(['AnimalType'], axis=1)
df1 = df1.drop(['Age'], axis=1)
df1 = df1.drop(['BreedType'], axis=1)
df1 = df1.drop(['FarmType'], axis=1)
df1 = df1.drop(['AnimalType&Age'], axis=1)

In [33]:
df1.head(50)

,SampleNumber,Date,Province,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,Lab_Reference,PigStage
0,184311,2023-01-03,Missing,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Missing
1,184312,2023-01-03,Girona,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
2,184329,2023-01-04,Barcelona,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing
3,184336,2023-01-04,Missing,Swine influenza virus,1,Tissue,PCR,Spain,Meat,4,Finishing
4,184337,2023-01-04,Lleida,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Finishing
5,184338,2023-01-04,Missing,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Finishing
6,184338,2023-01-04,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Finishing
7,184339,2023-01-04,Missing,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Finishing
8,184339,2023-01-04,Missing,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Finishing
9,184340,2023-01-04,Missing,Swine influenza virus,1,Tissue,PCR,Spain,Meat,4,Finishing


### Join both datasets

In [34]:
df_all = pd.concat([df, df1], ignore_index = True)

print(df_all['Pathogen'].unique())

['Mycoplasma hyopneumoniae' 'Swine influenza virus'
 'Porcine reproductive and respiratory syndrome virus' 'Brachyspira spp'
 'Actinobacillus pleuropneumoniae' 'Porcine epidemic diarrhea virus']


In [35]:
df_all.to_csv('output/PigLab4_update20250109.csv', index=False)

### Step 02: Create an RDF graph and namespaces.

In [37]:
g = rdflib.Graph()
xsd = Namespace('http://www.w3.org/2001/XMLSchema#')
g.bind('xsd', xsd)

# Define your custom namespace for your ontology's properties
LHO = Namespace("http://www.purl.org/decide/LiveStockHealthOnto/LHO#")
g.bind('LHO', LHO)

SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")
g.bind('skos', SKOS)

decide=  Namespace("http://www.purl.org/decide#")
g.bind('decide', decide)
ncit = Namespace("http://purl.obolibrary.org/obo/NCIT_C25464")
g.bind('ncit', ncit)
agrovoc = Namespace ("http://aims.fao.org/aos/agrovoc")
g.bind('agrovoc', agrovoc)

### Step 03: Iterate over the Panda DataFrame and map to ontology properties:

In [38]:
# for slugging local names or avoid spaces in names to create URIs along with label names

import re
import urllib.parse

def _safe_local_name(v):
    v = str(v).strip()
    # RFC3986 unreserved: A-Z a-z 0-9 - . _ ~
    v = re.sub(r'[^A-Za-z0-9\-._~]', '_', v)
    v = re.sub(r'_+', '_', v)  # collapse repeated underscores (optional)
    return v

In [39]:
#For Pig Data frame RDF conversion
#for index, row in barometer.head(500).iterrows():
##................................><.........................................................................
##Note: spaces in URIs are not OK. RDFLib (and the RDF specs) require valid URIs; a space makes them invalid. 
##If there is space in names then got errors in the Turtle serialization.
##................................><.........................................................................
for index, row in df_all.iterrows():
    SampleNumber = row["SampleNumber"]
    Date = row["Date"]
    Pathogen = row["Pathogen"]
    Breed = row["Breed"]
    Country = row["Country"]
    Province = row["Province"]
    SampleType = row["SampleType"]
    SampleResult = row["Result"]
    DiagnosticTest = row["DiagnosticTest"]
    PigStage= row["PigStage"]

    
    # Create a unique URI for each sample based on the row index
    PigSample_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#Lab1PigSample_{index}")
    
    # Add sample type assertion
    g.add((PigSample_uri, RDF.type, LHO.PigSample))
    description = "An individual representing a sample from a Pig and Piglets."
    g.add((PigSample_uri, RDFS.comment, Literal(description, lang="en")))

    
    #if "Pathogen" in row and row["Pathogen"]:    # Making little changes to pathogen URI so that it will not create error
        #Pathogen = row["Pathogen"]
        #Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen}")
        #g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        #g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        #description = "An individual representing Pig pathogen."
        #g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))

    if "Pathogen" in row and row["Pathogen"]:
        Pathogen = str(row["Pathogen"]).strip()
        Pathogen_local = _safe_local_name(Pathogen)  # e.g., "Swine_influenza_virus"
        Pathogen_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Pathogen_local}")
        g.add((PigSample_uri, LHO.hasPathogen, Pathogen_uri))
        g.add((Pathogen_uri, RDF.type, decide.Pathogen))
        description = "An individual representing Pig pathogen."
        g.add((Pathogen_uri, RDFS.comment, Literal(description, lang="en")))
        g.add((Pathogen_uri, RDFS.label, Literal(Pathogen)))  # human-readable conventional label, e.g., "Swine influenza virus" same thing we can do with other properties like breed and diagnostic test etc. if we have sapce

        
    if "Breed" in row and row["Breed"]:
        Breed = row["Breed"]
        Breed_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Breed}")
        g.add((PigSample_uri, LHO.hasBreed, Breed_uri))
        g.add((Breed_uri, RDF.type, LHO.Breed))
        description = "An individual representing a Piglet breed which is Meat."
        g.add((Breed_uri, RDFS.comment, Literal(description, lang="en")))

    if "Lab_Reference" in row and row["Lab_Reference"]:  #Adding that code
        Lab_Reference = row["Lab_Reference"]
        Lab_Reference_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Lab_Reference}")
        g.add((PigSample_uri, LHO.hasLab_Reference, Lab_Reference_uri))
        g.add((Lab_Reference_uri, RDF.type, LHO.Lab_Reference))
        description = "An individual representing from which lab refence it belongs."
        g.add((Lab_Reference_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "DiagnosticTest" in row and row["DiagnosticTest"]:
        DiagnosticTest= row["DiagnosticTest"]
        DiagnosticTest_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{DiagnosticTest}")
        g.add((PigSample_uri, LHO.hasDiagnosticTest, DiagnosticTest_uri))
        g.add((DiagnosticTest_uri, RDF.type, LHO.DiagnosticTest))
    if DiagnosticTest == "PCR":
        description = "An individual representing DNA/RNA amplification for rapid pathogen detection."
    elif SampleType == "Culture":
        description = "An individual representing a Growing live microorganisms for identification."
        g.add((DiagnosticTest_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Country" in row and row["Country"]:
        Country = row["Country"].strip()  # Use strip() to remove leading and trailing spaces
        # Encode the Country value to create a valid URI
        Country = urllib.parse.quote(Country)
        Country_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Country}")
    
        # Map to the specific class URI in your ontology and add the label
        g.add((PigSample_uri, LHO.hasCountry, Country_uri))
        g.add((Country_uri, RDF.type, URIRef("http://purl.obolibrary.org/obo/NCIT_C25464")))  # Use the specific class URI
        g.add((Country_uri, RDFS.label, Literal("Spain")))
        description = "An individual representing different Countries."
        g.add((Country_uri, RDFS.comment, Literal(description, lang="en")))
        
    if "Province" in row and row["Province"]:
        Province = row["Province"].strip()
        Province = urllib.parse.quote(Province)
        Province_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Province}")
        g.add((PigSample_uri, LHO.hasProvince, Province_uri))
        g.add((Province_uri, RDF.type, LHO.Province))
        description = "An individual representing different Province."
        g.add((Province_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "PigStage" in row and row["PigStage"]:
        Stage = row["PigStage"]
        Stage_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Stage}")
        g.add((PigSample_uri, LHO.hasProductionStages, Stage_uri))
        g.add((Stage_uri, RDF.type, LHO.PigProductionStage))
        description = "An individual representing Production stage ."
        g.add((Stage_uri, RDFS.comment, Literal(description, lang="en")))
    
    if "SampleType" in row and row["SampleType"]:
        SampleType = row["SampleType"]
        SampleType_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleType}")
        # Add statements for SampleType
        g.add((PigSample_uri, LHO.hasSampleType, SampleType_uri))
        g.add((SampleType_uri, RDF.type, URIRef("http://www.purl.org/decide#SampleType")))
    if SampleType == "Autopsy":
        description = "An individual representing a cattle sample obtained through autopsy."
    elif SampleType == "BAL":
        description = "An individual representing a cattle sample obtained through bronchoalveolar lavage (BAL)."
    elif SampleType == "SWAB":
        description = "An individual representing a cattle sample obtained through swabbing."
    elif SampleType == "Missing":
        description = "An individual representing a missing or unspecified cattle sample type."
        g.add((SampleType_uri, RDFS.comment, Literal(description, lang="en")))
           
    if "Result" in row and row["Result"]:
        SampleResult = row["Result"]
        # Create a unique URI for the Result based on the value
        Result_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{SampleResult}")
        g.add((PigSample_uri, LHO.hasResult, Result_uri))
        g.add((Result_uri, RDF.type, decide.SampleResult))
    if SampleResult == "1":
        description = "An individual representing a Positive test result"
    elif SampleResult == "0":
        description = "An individual representing a negative test result"
    elif SampleResult == "missing":
        description = "An individual representing an unknown or missing test result."
    g.add((Result_uri, RDFS.comment, Literal(description, lang="en")))


    if "Date" in row and not pd.isnull(row["Date"]):  # Check for NaN or None values
        Date = str(row["Date"]).strip()
    
    # Split the datetime string and take the date part
        Date = Date.split()[0]
    
    # Create a unique URI for the Date based on the value (use the Date directly)
        Date_uri = URIRef(f"http://www.purl.org/decide/LiveStockHealthOnto/LHO#{Date}")
    
    # Add Date assertion
        g.add((PigSample_uri, LHO.hasDate, Date_uri))
        g.add((Date_uri, RDF.type, LHO.Date))
    

# Serialize the RDF graph to a file
rdf_output_file = "output/PigLab1.rdf"
g.serialize(rdf_output_file, format="xml")

# Serialize the RDF graph to Turtle format and print it too see RDF conversion
#turtle_data = g.serialize(format="turtle")
#print(turtle_data)

<Graph identifier=Nddebdda5e7d841b1a1ae01723fbcc080 (<class 'rdflib.graph.Graph'>)>

### Step 4: For Knowledge Graph,Load the RDF data and ontology into a Panda DataFrame:

In [40]:
# Parse the ontology file in OWL format and add it to the graph
path_to_ontology = "Ontology/LivestockHealthOntology.owl"
g.parse(path_to_ontology, format="xml")

<Graph identifier=Nddebdda5e7d841b1a1ae01723fbcc080 (<class 'rdflib.graph.Graph'>)>

### Step 05: Query the data from updated ontology

In [41]:
query = """
PREFIX rdf:   <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX decide: <http://www.purl.org/decide#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX LHO:   <http://www.purl.org/decide/LiveStockHealthOnto/LHO#>
PREFIX skos:  <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd:   <http://www.w3.org/2001/XMLSchema#>

SELECT
  (STRAFTER(STR(?Sample), "#") AS ?SampleNumber)
  (STRAFTER(STR(?Province), "#") AS ?Province)
  (?FlooredDate AS ?FlooredDate)
  (COALESCE(STR(?PathogenLabel), REPLACE(STRAFTER(STR(?Pathogen), "#"), "_", " ")) AS ?Pathogen)
  (STRAFTER(STR(?SampleResult), "#") AS ?Result)
  (STRAFTER(STR(?SampleType), "#") AS ?SampleType)
  (STRAFTER(STR(?DiagnosticTest), "#") AS ?DiagnosticTest)
  (STRAFTER(STR(?Country), "#") AS ?Country)
  (STRAFTER(STR(?Breed), "#") AS ?Breed)
  (STRAFTER(STR(?Lab_Reference), "#") AS ?Lab_Reference)
  (STRAFTER(STR(?PigProductionStages), "#") AS ?PigStage)
  (STRAFTER(STR(?AutonomousCommunity), "#") AS ?AutonomousCommunity)
  (?MonthNum AS ?Month)
  (?YearNum  AS ?Year)
WHERE {
  ?Sample rdf:type LHO:PigSample .
  ?Sample LHO:hasDate ?Date .
  ?Sample LHO:hasPathogen ?Pathogen .
  ?Sample LHO:hasBreed ?Breed .
  ?Sample LHO:hasSampleType ?SampleType .
  ?Sample LHO:hasDiagnosticTest ?DiagnosticTest .
  ?Sample LHO:hasProductionStages ?PigProductionStages .
  ?Sample LHO:hasProvince ?Province .
  ?Sample LHO:hasCountry ?Country .
  ?Sample LHO:hasResult ?SampleResult .
  ?Sample LHO:hasLab_Reference ?Lab_Reference .
  OPTIONAL { ?Sample LHO:hasAutonomousCommunity ?AutonomousCommunity . }

  # Pathogen label only (prefer rdfs:label/skos:prefLabel; prefer en or no-lang)
  OPTIONAL {
    ?Pathogen (rdfs:label|skos:prefLabel) ?PathogenLabel .
    FILTER(LANG(?PathogenLabel) = "" || LANGMATCHES(LANG(?PathogenLabel), "en"))
  }

  # ---- Date parsing (supports DD/MM/YYYY and YYYY-MM-DD) ----
  BIND( STRAFTER(STR(?Date), "#") AS ?DateStr )
  BIND( ?DateStr AS ?FlooredDate )

  # month
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 4, 2),   # DD/MM/YYYY -> MM is positions 4-5
         SUBSTR(?DateStr, 6, 2)    # YYYY-MM-DD -> MM is positions 6-7
      )
    ) AS ?MonthNum
  )

  # year
  BIND(
    xsd:integer(
      IF(CONTAINS(?DateStr, "/"),
         SUBSTR(?DateStr, 7, 4),   # DD/MM/YYYY -> YYYY starts at 7
         SUBSTR(?DateStr, 1, 4)    # YYYY-MM-DD -> YYYY is positions 1-4
      )
    ) AS ?YearNum
  )
}
"""
results = g.query(query)

data = []
for row in results:
    data.append(list(row))

df = pd.DataFrame(data, columns=[
    "SampleNumber","Province","FlooredDate","Pathogen","Result","SampleType",
    "DiagnosticTest","Country","Breed","LabReference","PigStage",
    "AutonomousCommunity","Month","Year"
])

# Display the dataframe
df

,SampleNumber,Province,FlooredDate,Pathogen,Result,SampleType,DiagnosticTest,Country,Breed,LabReference,PigStage,AutonomousCommunity,Month,Year
0,Lab1PigSample_0,Missing,2020-07-01,Mycoplasma hyopneumoniae,0,Tissue,PCR,Spain,Meat,4,Missing,None,7,2020
1,Lab1PigSample_1,Missing,2020-07-01,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing,None,7,2020
2,Lab1PigSample_2,Missing,2020-07-01,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing,None,7,2020
3,Lab1PigSample_3,Missing,2020-07-01,Swine influenza virus,0,Tissue,PCR,Spain,Meat,4,Missing,None,7,2020
4,Lab1PigSample_4,Missing,2020-07-01,Mycoplasma hyopneumoniae,1,Tissue,PCR,Spain,Meat,4,Missing,None,7,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134721,Lab1PigSample_134721,Zaragoza,2025-03-13,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,4,Weaning,None,3,2025
134722,Lab1PigSample_134722,Zaragoza,2025-03-13,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,4,Weaning,None,3,2025
134723,Lab1PigSample_134723,Zaragoza,2025-03-13,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,4,Weaning,None,3,2025
134724,Lab1PigSample_134724,Zaragoza,2025-03-13,Porcine reproductive and respiratory syndrome ...,0,Blood,PCR,Spain,Meat,4,Weaning,None,3,2025


In [43]:
# Save Results: save to CSV
import os
os.makedirs("output", exist_ok=True)

df.to_csv("output/PigLab4_query_results.csv", index=False, encoding="utf-8")
print("Saved: output/PigLab4_query_results_baromterepig.csv")

Saved: output/PigLab4_query_results_baromterepig.csv
